In [5]:
I2cm_lattice= '''
5.4016255140719478    0.0000000000000000    0.0000000000000000
0.0000000000000000    5.4310978228412008    0.0000000000000000
0.0000000000000000    0.0000000000000000   22.1176750326256908
'''

# Splitting the data into rows and then into elements, converting each to a float
I2cm_lattice = [list(map(float, row.split())) for row in I2cm_lattice.strip().split('\n')]

# Converting the data to a NumPy matrix
I2cm_lattice = np.array(I2cm_lattice)

5.4562230143487103 0.0000000000000000 0.0000000000000000
0.0000000000000000 5.4562230143487103 0.0000000000000000
0.0000000000000000 0.0000000000000000 22.2110543204440276


In [6]:
import numpy as np
# Input data as a multiline string
primitive_lattice = """
3.8581322931120781    0.0000000000000000    0.0000000000000000
0.0000000000000000    3.8581322931120781    0.0000000000000000
0.0000000000000000    0.0000000000000000   11.1055271602220138
"""

# Splitting the data into rows and then into elements, converting each to a float
primitive_lattice = [list(map(float, row.split())) for row in primitive_lattice.strip().split('\n')]

# Converting the data to a NumPy matrix
primitive_lattice = np.array(primitive_lattice)

transformation_matrix = np.array([
                                    [1, 1, 0],
                                    [1, -1, 0],
                                    [0, 0, 2]
                                ])

transformation_matrix = np.array([
    [np.sqrt(2), 0, 0],
    [0, np.sqrt(2), 0],
    [0, 0, 2]
])


# Performing the matrix multiplication
lattice_basis = np.dot(transformation_matrix, primitive_lattice)
# Transformation matrix t


def matrix_to_string(matrix):
    # Converting each row of the matrix to a string
    row_strings = [" ".join(f"{value:.16f}" for value in row) for row in matrix]

    # Joining all row strings with a newline character
    return "\n".join(row_strings)

# Convert the matrix to a string in the desired format
formatted_string = matrix_to_string(lattice_basis)

print(formatted_string)

5.4562230143487103 0.0000000000000000 0.0000000000000000
0.0000000000000000 5.4562230143487103 0.0000000000000000
0.0000000000000000 0.0000000000000000 22.2110543204440276


In [7]:
import numpy as np

# Input data as a multiline string, including labels
data_str = """
Rb  0.0000000000000000  0.0000000000000000  0.5000000000000000
Nd  0.0000000000000000  0.0000000000000000 -0.0000000000000000
Nb  0.5000000000000000  0.5000000000000000  0.2005811844185174
Nb  0.5000000000000000  0.5000000000000000  0.7994187855814798
O  0.0000000000000000  0.5000000000000000  0.1608942385069339
O -0.0000000000000000  0.5000000000000000  0.8391057614930659
O  0.5000000000000000  0.0000000000000000  0.1608942385069339
O  0.5000000000000000 -0.0000000000000000  0.8391057614930659
O  0.5000000000000000  0.5000000000000000  0.3616055463234930
O  0.5000000000000000  0.5000000000000000  0.6383944836765096
O  0.5000000000000000  0.5000000000000000 -0.000000000000000
"""

def transformation(data_str):
    # Splitting the data into rows, separating labels from numerical data
    labels, numerical_data = [], []
    for row in data_str.strip().split('\n'):
        elements = row.split()
        labels.append(elements[0])  # Extract label
        numerical_data.append(list(map(float, elements[1:])))  # Convert remaining elements to floats

    # Converting the numerical data to a NumPy matrix
    matrix = np.array(numerical_data)

    # Transformation matrix t
    transformation_matrix = np.array([
        [1, 1, 0],
        [1, -1, 0],
        [0, 0, 2]
    ])

    # Calculating the inverse of the transformation matrix
    inverse_t = np.linalg.inv(transformation_matrix)

    # Performing the matrix multiplication
    result = np.dot(inverse_t, matrix.T).T

    # Transpose back and reattach the labels
    result_labeled = np.column_stack((labels, result))
    return result, result_labeled

result, result_labeled = transformation(data_str)

label_to_data = {}

for row in result_labeled:
    elements = row
    label = elements[0]  # Extract label
    numerical_values = list(map(float, elements[1:]))  # Convert remaining elements to floats
    # Add numerical data to corresponding label list
    if label in label_to_data:
        label_to_data[label].append(numerical_values)
    else:
        label_to_data[label] = [numerical_values]

# Print unique labels
print("Unique Labels:", label_to_data.keys())

# Access data for each label
#for label, data in label_to_data.items():
 #   print(f"Data for label '{label}':")
 #   for item in data:
  #      print(item)


def shifting(matrix):
    # Apply the first transformation (add 0.5 to the third column)
    transformed_matrix_z = np.copy(matrix)
    transformed_matrix_z[:, 2] += 0.5  # Adjusted index for mixed type

    # Intertwine the rows from the first transformation
    intertwined_matrix_z = np.vstack([np.vstack([row, transformed_row]) for row, transformed_row in zip(matrix, transformed_matrix_z)])

    # Apply the second transformation (add 0.5 to the first and second columns)
    def second_transformation(row):
        new_row = np.copy(row)
        new_row[0] += 0.5  # Adjusted index for mixed type
        new_row[1] += 0.5  # Adjusted index for mixed type
        return new_row

    intertwined_matrix_xy = np.vstack([np.vstack([row, second_transformation(row)]) for row in intertwined_matrix_z])
    intertwined_matrix_xy = intertwined_matrix_xy%1
    return intertwined_matrix_xy

# Apply the transformation and intertwine the rows
P4mmm_matrix = np.vstack([shifting(label_to_data[lables]) for lables in label_to_data.keys()])


# Apply the transformation and intertwine the rows
transformed_arrays = []
for label in label_to_data.keys():
    transformed_data = shifting(label_to_data[label])

    # Prepend each row with the label
    labeled_data = np.hstack([[[label]] * transformed_data.shape[0], transformed_data])  # Repeat label for each row
    transformed_arrays.append(labeled_data)

# Stack the transformed arrays vertically
labeled_matrix = np.vstack(transformed_arrays)

def matrix_to_string(matrix):
    # Update this function to handle labels correctly
    row_strings = [" ".join(str(value) for value in row) for row in matrix]
    return "\n".join(row_strings)

# Convert the matrix to a string in the desired format
#formatted_string = matrix_to_string(labeled_matrix)
formatted_num = matrix_to_string(P4mmm_matrix)
#print(formatted_string)
#print(len(labeled_matrix))
print(formatted_num)

Unique Labels: dict_keys(['Rb', 'Nd', 'Nb', 'O'])
0.0 0.0 0.25
0.5 0.5 0.25
0.0 0.0 0.75
0.5 0.5 0.75
0.0 0.0 0.0
0.5 0.5 0.0
0.0 0.0 0.5
0.5 0.5 0.5
0.5 0.0 0.1002905922092587
0.0 0.5 0.1002905922092587
0.5 0.0 0.6002905922092587
0.0 0.5 0.6002905922092587
0.5 0.0 0.3997093927907399
0.0 0.5 0.3997093927907399
0.5 0.0 0.8997093927907399
0.0 0.5 0.8997093927907399
0.25 0.75 0.08044711925346695
0.75 0.25 0.08044711925346695
0.25 0.75 0.5804471192534669
0.75 0.25 0.5804471192534669
0.25 0.75 0.41955288074653296
0.75 0.25 0.41955288074653296
0.25 0.75 0.919552880746533
0.75 0.25 0.919552880746533
0.25 0.25 0.08044711925346695
0.75 0.75 0.08044711925346695
0.25 0.25 0.5804471192534669
0.75 0.75 0.5804471192534669
0.25 0.25 0.41955288074653296
0.75 0.75 0.41955288074653296
0.25 0.25 0.919552880746533
0.75 0.75 0.919552880746533
0.5 0.0 0.1808027731617465
0.0 0.5 0.1808027731617465
0.5 0.0 0.6808027731617465
0.0 0.5 0.6808027731617465
0.5 0.0 0.3191972418382548
0.0 0.5 0.3191972418382548
0.5 

In [3]:
import numpy as np
# Input data as a multiline string
A2plus_dist = """
   0.00000000  0.00000000  0.25000000
   0.50000000  0.50000000  0.25000000
   0.00000000  0.00000000  0.75000000
   0.50000000  0.50000000  0.75000000
   0.00000000  0.00000000  0.00000000
   0.50000000  0.50000000  0.00000000
   0.00000000  0.00000000  0.50000000
   0.50000000  0.50000000  0.50000000
   0.50000000  0.00000000  0.10324211
   0.00000000  0.50000000  0.10324211
   0.50000000  0.00000000  0.60324211
   0.00000000  0.50000000  0.60324211
   0.50000000  0.00000000  0.39675787
   0.00000000  0.50000000  0.39675787
   0.50000000  0.00000000  0.89675787
   0.00000000  0.50000000  0.89675787
   0.29800502  0.70199498  0.08447320
   0.70199498  0.29800502  0.08447320
   0.20199498  0.79800502  0.58447320
   0.79800502  0.20199498  0.58447320
   0.20199498  0.79800502  0.41552680
   0.79800502  0.20199498  0.41552680
   0.29800502  0.70199498  0.91552680
   0.70199498  0.29800502  0.91552680
   0.20199498  0.20199498  0.08447320
   0.79800502  0.79800502  0.08447320
   0.29800502  0.29800502  0.58447320
   0.70199498  0.70199498  0.58447320
   0.29800502  0.29800502  0.41552680
   0.70199498  0.70199498  0.41552680
   0.20199498  0.20199498  0.91552680
   0.79800502  0.79800502  0.91552680
   0.50000000  0.00000000  0.18226416
   0.00000000  0.50000000  0.18226416
   0.50000000  0.00000000  0.68226416
   0.00000000  0.50000000  0.68226416
   0.50000000  0.00000000  0.31773586
   0.00000000  0.50000000  0.31773586
   0.50000000  0.00000000  0.81773586
   0.00000000  0.50000000  0.81773586
   0.50000000  0.00000000  0.00000000
   0.00000000  0.50000000  0.00000000
   0.50000000  0.00000000  0.50000000
   0.00000000  0.50000000  0.50000000
"""

# Splitting the data into rows and then into elements, converting each to a float
A2plus_dist = [list(map(float, row.split())) for row in A2plus_dist.strip().split('\n')]

# Converting the data to a NumPy matrix
A2plus_dist = np.array(A2plus_dist)

In [24]:
import numpy as np
# Input data as a multiline string
A5minus_dist = """
  0.0000000000000000  0.0000000000000000  0.2500000000000000
  0.5000000000000000  0.5000000000000000  0.2500000000000000
  0.0000000000000000  0.0000000000000000  0.7500000000000000
  0.5000000000000000  0.5000000000000000  0.7500000000000000
  0.0101556972960779  0.9999999999950759  0.0000000000000000
  0.4898443027039221  0.5000000000049241  0.0000000000000000
  0.9898443027039221  0.0000000000049241  0.5000000000000000
  0.5101556972960779  0.4999999999950759  0.5000000000000000
  0.5049328348286792  0.9999999999969376  0.1011065439142769
  0.9950671651713208  0.5000000000030624  0.1011065439142769
  0.4950671651713208  0.0000000000030624  0.6011065439142840
  0.0049328348286792  0.4999999999969376  0.6011065439142840
  0.4950671651713208  0.0000000000030624  0.3988934410857183
  0.0049328348286792  0.4999999999969376  0.3988934410857183
  0.5049328348286792  0.9999999999969376  0.8988934410857183
  0.9950671651713208  0.5000000000030624  0.8988934410857183
  0.2500000000000000  0.7500000000000000  0.0945840435275898
  0.7500000000000000  0.2500000000000000  0.0707233254047139
  0.2500000000000000  0.7500000000000000  0.5707233254047139
  0.7500000000000000  0.2500000000000000  0.5945840435275898
  0.2500000000000000  0.7500000000000000  0.4292766745952861
  0.7500000000000000  0.2500000000000000  0.4054159564724102
  0.2500000000000000  0.7500000000000000  0.9054159564724102
  0.7500000000000000  0.2500000000000000  0.9292766745952861
  0.2500000000000000  0.2500000000000000  0.0945840435275898
  0.7500000000000000  0.7500000000000000  0.0707233254047139
  0.2500000000000000  0.2500000000000000  0.5707233254047139
  0.7500000000000000  0.7500000000000000  0.5945840435275898
  0.2500000000000000  0.2500000000000000  0.4292766745952861
  0.7500000000000000  0.7500000000000000  0.4054159564724102
  0.2500000000000000  0.2500000000000000  0.9054159564724102
  0.7500000000000000  0.7500000000000000  0.9292766745952861
  0.5622188362726348  0.0000000000000000  0.1809011193235435
  0.9377811637273652  0.5000000000000000  0.1809011193235435
  0.4377811637273652  0.0000000000000000  0.6809011193235435
  0.0622188362726348  0.5000000000000000  0.6809011193235435
  0.4377811637273652  0.0000000000000000  0.3190988956764542
  0.0622188362726348  0.5000000000000000  0.3190988956764542
  0.5622188362726348  0.0000000000000000  0.8190988956764542
  0.9377811637273652  0.5000000000000000  0.8190988956764542
  0.4292630702238611  0.0000000000035385  0.0000000000000000
  0.0707369297761389  0.4999999999964615  0.0000000000000000
  0.5707369297761389  0.9999999999964615  0.5000000000000000
  0.9292630702238611  0.5000000000035385  0.5000000000000000
"""

# Splitting the data into rows and then into elements, converting each to a float
A5minus_dist = [list(map(float, row.split())) for row in A5minus_dist.strip().split('\n')]

# Converting the data to a NumPy matrix
A5minus_dist = np.array(A5minus_dist)

In [25]:
P4mmm_dist = '''
0.0 0.0 0.25
0.5 0.5 0.25
0.0 0.0 0.75
0.5 0.5 0.75
0.0 0.0 0.0
0.5 0.5 0.0
0.0 0.0 0.5
0.5 0.5 0.5
0.5 0.0 0.1002905922092587
0.0 0.5 0.1002905922092587
0.5 0.0 0.6002905922092587
0.0 0.5 0.6002905922092587
0.5 0.0 0.3997093927907399
0.0 0.5 0.3997093927907399
0.5 0.0 0.8997093927907399
0.0 0.5 0.8997093927907399
0.25 0.75 0.08044711925346695
0.75 0.25 0.08044711925346695
0.25 0.75 0.5804471192534669
0.75 0.25 0.5804471192534669
0.25 0.75 0.41955288074653296
0.75 0.25 0.41955288074653296
0.25 0.75 0.919552880746533
0.75 0.25 0.919552880746533
0.25 0.25 0.08044711925346695
0.75 0.75 0.08044711925346695
0.25 0.25 0.5804471192534669
0.75 0.75 0.5804471192534669
0.25 0.25 0.41955288074653296
0.75 0.75 0.41955288074653296
0.25 0.25 0.919552880746533
0.75 0.75 0.919552880746533
0.5 0.0 0.1808027731617465
0.0 0.5 0.1808027731617465
0.5 0.0 0.6808027731617465
0.0 0.5 0.6808027731617465
0.5 0.0 0.3191972418382548
0.0 0.5 0.3191972418382548
0.5 0.0 0.8191972418382548
0.0 0.5 0.8191972418382548
0.5 0.0 0.0
0.0 0.5 0.0
0.5 0.0 0.5
0.0 0.5 0.5
'''

# Splitting the data into rows and then into elements, converting each to a float
P4mmm_dist = [list(map(float, row.split())) for row in P4mmm_dist.strip().split('\n')]

# Converting the data to a NumPy matrix
P4mmm_dist = np.array(P4mmm_dist)

In [15]:
P4mmm_prim = """
0.0000000000000000  0.0000000000000000  0.5000000000000000
0.0000000000000000  0.0000000000000000 -0.0000000000000000
0.5000000000000000  0.5000000000000000  0.2005811844185174
0.5000000000000000  0.5000000000000000  0.7994187855814798
0.0000000000000000  0.5000000000000000  0.1608942385069339
-0.0000000000000000  0.5000000000000000  0.8391057614930659
0.5000000000000000  0.0000000000000000  0.1608942385069339
0.5000000000000000 -0.0000000000000000  0.8391057614930659
0.5000000000000000  0.5000000000000000  0.3616055463234930
0.5000000000000000  0.5000000000000000  0.6383944836765096
0.5000000000000000  0.5000000000000000 -0.000000000000000
"""

# Splitting the data into rows and then into elements, converting each to a float
P4mmm_prim = [list(map(float, row.split())) for row in P4mmm_prim.strip().split('\n')]

# Converting the data to a NumPy matrix
P4mmm_prim = np.array(P4mmm_prim)

In [8]:
I2cm = """
  0.0000000000000000  0.0253598435636349  0.2500000000000000
  0.5000000000000000  0.5253598435636349  0.2500000000000000
  0.0000000000000000  0.0253598435636349  0.7500000000000000
  0.5000000000000000  0.5253598435636349  0.7500000000000000
  0.9987341325845875  0.0405767565133530  0.0000000000000000
  0.5012658674154125  0.5405767565133530  0.0000000000000000
  0.0012658674154125  0.0405767565133530  0.5000000000000000
  0.4987341325845875  0.5405767565133530  0.5000000000000000
  0.4994073282184956  0.0025311698215447  0.1025951201243132
  0.0005926717815044  0.5025311698215447  0.1025951201243132
  0.5005926717815044  0.0025311698215447  0.6025951201243203
  0.9994073282184956  0.5025311698215447  0.6025951201243203
  0.5005926717815044  0.0025311698215447  0.3974048648756821
  0.9994073282184956  0.5025311698215447  0.3974048648756821
  0.4994073282184956  0.0025311698215447  0.8974048648756821
  0.0005926717815044  0.5025311698215447  0.8974048648756821
  0.3066333364250582  0.6873281529321744  0.0971680805091282
  0.7164670553360821  0.2750605216016240  0.0730165145191606
  0.2164670553360821  0.7750605216016240  0.5730165145191606
  0.8066333364250653  0.1873281529321673  0.5971680805091211
  0.2164670553360821  0.7750605216016240  0.4269834854808394
  0.8066333364250653  0.1873281529321673  0.4028319194908718
  0.3066333364250582  0.6873281529321744  0.9028319194908789
  0.7164670553360821  0.2750605216016240  0.9269834854808394
  0.1933666635749418  0.1873281529321673  0.0971680805091353
  0.7835329446639179  0.7750605216016240  0.0730165145191535
  0.2835329446639179  0.2750605216016240  0.5730165145191535
  0.6933666635749347  0.6873281529321744  0.5971680805091282
  0.2835329446639179  0.2750605216016240  0.4269834854808465
  0.6933666635749347  0.6873281529321744  0.4028319194908647
  0.1933666635749418  0.1873281529321673  0.9028319194908718
  0.7835329446639179  0.7750605216016240  0.9269834854808465
  0.5550072664298966  0.0133304477798646  0.1816590605376689
  0.9449927335701034  0.5133304477798646  0.1816590605376689
  0.4449927335701034  0.0133304477798646  0.6816590605376689
  0.0550072664298966  0.5133304477798646  0.6816590605376689
  0.4449927335701034  0.0133304477798646  0.3183409544623288
  0.0550072664298966  0.5133304477798646  0.3183409544623288
  0.5550072664298966  0.0133304477798646  0.8183409544623288
  0.9449927335701034  0.5133304477798646  0.8183409544623288
  0.4266945171391043  0.9776432148154726  0.0000000000000000
  0.0733054828608957  0.4776432148154726  0.0000000000000000
  0.5733054828608957  0.9776432148154726  0.5000000000000000
  0.9266945171391043  0.4776432148154726  0.5000000000000000
"""

# Splitting the data into rows and then into elements, converting each to a float
I2cm = [list(map(float, row.split())) for row in I2cm.strip().split('\n')]

# Converting the data to a NumPy matrix
I2cm = np.array(I2cm)

In [16]:
GM5m_dist = '''
0.0033831149557244  0.0033831149557244  0.5000000000000000
0.0269069565823017  0.0269069565823017  0.0000000000000000
0.5029097240421265  0.5029097240421265  0.2008918145076706
0.5029097240421265  0.5029097240421265  0.7991081554923269
0.9981589185041102  0.4951081267560369  0.1616127375974443
0.9981589426920436  0.4951081025681034  0.8383872624025557
0.4951081267560369  0.9981589185041102  0.1616127375974443
0.4951081025681034  0.9981589426920436  0.8383872624025557
0.4968035955369672  0.4968035955369672  0.3618132956874902
0.4968035955369672  0.4968035955369672  0.6381867343125123
0.4838270780062643  0.4838270780062643  0.0000000000000000
'''

# Splitting the data into rows and then into elements, converting each to a float
GM5m_dist = [list(map(float, row.split())) for row in GM5m_dist.strip().split('\n')]

# Converting the data to a NumPy matrix
GM5m_dist = np.array(GM5m_dist)

In [25]:
A2plus = '''
hhh
O3    0.25005  0.75005  0.58143 -1.0000  1.0000  0.0000 
O1    0.25005  0.75005  0.08143  1.0000 -1.0000  0.0000 
O4    0.75005  0.25005  0.58143  1.0000 -1.0000  0.0000 
O2    0.75005  0.25005  0.08143 -1.0000  1.0000  0.0000 
O6    0.75005  0.25005  0.41867  1.0000 -1.0000  0.0000 
O8    0.75005  0.25005  0.91867 -1.0000  1.0000  0.0000 
O5    0.25005  0.75005  0.41867 -1.0000  1.0000  0.0000 
O7    0.25005  0.75005  0.91867  1.0000 -1.0000  0.0000 
O13   0.25005  0.25005  0.41867  1.0000  1.0000  0.0000 
O15   0.25005  0.25005  0.91867 -1.0000 -1.0000  0.0000 
O14   0.75005  0.75005  0.41867 -1.0000 -1.0000  0.0000 
O16   0.75005  0.75005  0.91867  1.0000  1.0000  0.0000 
O11   0.25005  0.25005  0.58143  1.0000  1.0000  0.0000 
O9    0.25005  0.25005  0.08143 -1.0000 -1.0000  0.0000 
O12   0.75005  0.75005  0.58143 -1.0000 -1.0000  0.0000 
O10   0.75005  0.75005  0.08143  1.0000  1.0000  0.0000 
'''


In [4]:
A5minus_eigenvec = '''
0 0 0
0 0 0
0 0 0
0 0 0
0.2045219399607055 6.722480503321745e-15 0
-0.2045219399607055 -6.722480503321745e-15 0
-0.2045219399607055 -6.722480503321745e-15 0
0.2045219399607055 6.722480503321745e-15 0
0.02571228125277871 -8.475620083108975e-16 0
-0.02571228125277871 8.475620083108975e-16 0
-0.02571228125277871 8.475620083108975e-16 0
0.02571228125277871 -8.475620083108975e-16 0
-0.02571228125277871 8.475620083108975e-16 0
0.02571228125277871 -8.475620083108975e-16 0
0.02571228125277871 -8.475620083108975e-16 0
-0.02571228125277871 8.475620083108975e-16 0
0 0 0.133151399655914
0 0 -0.133151399655914
0 0 -0.133151399655914
0 0 0.133151399655914
0 0 0.133151399655914
0 0 -0.133151399655914
0 0 -0.133151399655914
0 0 0.133151399655914
0 0 0.1331513996559229
0 0 -0.1331513996559229
0 0 -0.1331513996559229
0 0 0.1331513996559229
0 0 0.1331513996559229
0 0 -0.1331513996559229
0 0 -0.1331513996559229
0 0 0.1331513996559229
0.07660672759536045 -2.582096897444975e-15 0
-0.07660672759536045 2.582096897444975e-15 0
-0.07660672759536045 2.582096897444975e-15 0
0.07660672759536045 -2.582096897444975e-15 0
-0.07660672759536045 2.582096897444975e-15 0
0.07660672759536045 -2.582096897444975e-15 0
0.07660672759536045 -2.582096897444975e-15 0
-0.07660672759536045 2.582096897444975e-15 0
-0.3524119335567476 1.173952250160603e-14 0
0.3524119335567476 -1.173952250160603e-14 0
0.3524119335567476 -1.173952250160603e-14 0
-0.3524119335567476 1.173952250160603e-14 0

0 0 0
0 0 0
0 0 0
0 0 0
-6.790556393219222e-15 0.2044965361985659 0
6.790556393219222e-15 -0.2044965361985659 0
6.790556393219222e-15 -0.2044965361985659 0
-6.790556393219222e-15 0.2044965361985659 0
-8.543618475966918e-16 -0.02573482231624323 0
8.543618475966918e-16 0.02573482231624323 0
8.543618475966918e-16 0.02573482231624323 0
-8.543618475966918e-16 -0.02573482231624323 0
8.543618475966918e-16 0.02573482231624323 0
-8.543618475966918e-16 -0.02573482231624323 0
-8.543618475966918e-16 -0.02573482231624323 0
8.543618475966918e-16 0.02573482231624323 0
0 0 -0.1331561972619213
0 0 0.1331561972619213
0 0 0.1331561972619213
0 0 -0.1331561972619213
0 0 -0.1331561972619213
0 0 0.1331561972619213
0 0 0.1331561972619213
0 0 -0.1331561972619213
0 0 0.1331561972619124
0 0 -0.1331561972619124
0 0 -0.1331561972619124
0 0 0.1331561972619124
0 0 0.1331561972619124
0 0 -0.1331561972619124
0 0 -0.1331561972619124
0 0 0.1331561972619124
-2.570242005109214e-15 -0.0767760802380166 0
2.570242005109214e-15 0.0767760802380166 0
2.570242005109214e-15 0.0767760802380166 0
-2.570242005109214e-15 -0.0767760802380166 0
2.570242005109214e-15 0.0767760802380166 0
-2.570242005109214e-15 -0.0767760802380166 0
-2.570242005109214e-15 -0.0767760802380166 0
2.570242005109214e-15 0.0767760802380166 0
1.173210825792834e-14 0.3523424187534689 0
-1.173210825792834e-14 -0.3523424187534689 0
-1.173210825792834e-14 -0.3523424187534689 0
1.173210825792834e-14 0.3523424187534689 0

0 0 0
0 0 0
0 0 0
0 0 0
2.718372739579425e-14 -0.1960338230930462 0
-2.718372739579425e-14 0.1960338230930462 0
-2.718372739579425e-14 0.1960338230930462 0
2.718372739579425e-14 -0.1960338230930462 0
1.060309917240831e-16 0.001095206513896322 0
-1.060309917240831e-16 -0.001095206513896322 0
-1.060309917240831e-16 -0.001095206513896322 0
1.060309917240831e-16 0.001095206513896322 0
-1.060309917240831e-16 -0.001095206513896322 0
1.060309917240831e-16 0.001095206513896322 0
1.060309917240831e-16 0.001095206513896322 0
-1.060309917240831e-16 -0.001095206513896322 0
0 0 -0.06405785239828143
0 0 0.06405785239828143
0 0 0.06405785239828143
0 0 -0.06405785239828143
0 0 -0.06405785239828143
0 0 0.06405785239828143
0 0 0.06405785239828143
0 0 -0.06405785239828143
0 0 0.06405785239826367
0 0 -0.06405785239826367
0 0 -0.06405785239826367
0 0 0.06405785239826367
0 0 0.06405785239826367
0 0 -0.06405785239826367
0 0 -0.06405785239826367
0 0 0.06405785239826367
-4.179585849676672e-14 -0.3017200325527394 0
4.179585849676672e-14 0.3017200325527394 0
4.179585849676672e-14 0.3017200325527394 0
-4.179585849676672e-14 -0.3017200325527394 0
4.179585849676672e-14 0.3017200325527394 0
-4.179585849676672e-14 -0.3017200325527394 0
-4.179585849676672e-14 -0.3017200325527394 0
4.179585849676672e-14 0.3017200325527394 0
-1.576286713897427e-14 -0.1143885979726138 0
1.576286713897427e-14 0.1143885979726138 0
1.576286713897427e-14 0.1143885979726138 0
-1.576286713897427e-14 -0.1143885979726138 0

0 0 0
0 0 0
0 0 0
0 0 0
0.1957395377765869 2.71422629247235e-14 0
-0.1957395377765869 -2.71422629247235e-14 0
-0.1957395377765869 -2.71422629247235e-14 0
0.1957395377765869 2.71422629247235e-14 0
0.001036835847520322 -1.537646758163927e-16 0
-0.001036835847520322 1.537646758163927e-16 0
-0.001036835847520322 1.537646758163927e-16 0
0.001036835847520322 -1.537646758163927e-16 0
-0.001036835847520322 1.537646758163927e-16 0
0.001036835847520322 -1.537646758163927e-16 0
0.001036835847520322 -1.537646758163927e-16 0
-0.001036835847520322 1.537646758163927e-16 0
0 0 -0.06396026968639698
0 0 0.06396026968639698
0 0 0.06396026968639698
0 0 -0.06396026968639698
0 0 -0.06396026968639698
0 0 0.06396026968639698
0 0 0.06396026968639698
0 0 -0.06396026968639698
0 0 -0.0639602696864148
0 0 0.0639602696864148
0 0 0.0639602696864148
0 0 -0.0639602696864148
0 0 -0.0639602696864148
0 0 0.0639602696864148
0 0 0.0639602696864148
0 0 -0.0639602696864148
-0.3018988680105122 4.18579108623247e-14 0
0.3018988680105122 -4.18579108623247e-14 0
0.3018988680105122 -4.18579108623247e-14 0
-0.3018988680105122 4.18579108623247e-14 0
0.3018988680105122 -4.18579108623247e-14 0
-0.3018988680105122 4.18579108623247e-14 0
-0.3018988680105122 4.18579108623247e-14 0
0.3018988680105122 -4.18579108623247e-14 0
-0.1141681474333813 1.581655470476785e-14 0
0.1141681474333813 -1.581655470476785e-14 0
0.1141681474333813 -1.581655470476785e-14 0
-0.1141681474333813 1.581655470476785e-14 0

0 0 0
0 0 0
0 0 0
0 0 0
2.087751585457967e-13 -0.3393175885211503 0
-2.087751585457967e-13 0.3393175885211503 0
-2.087751585457967e-13 0.3393175885211503 0
2.087751585457967e-13 -0.3393175885211503 0
4.21384749990479e-14 0.06846776108008019 0
-4.21384749990479e-14 -0.06846776108008019 0
-4.21384749990479e-14 -0.06846776108008019 0
4.21384749990479e-14 0.06846776108008019 0
-4.21384749990479e-14 -0.06846776108008019 0
4.21384749990479e-14 0.06846776108008019 0
4.21384749990479e-14 0.06846776108008019 0
-4.21384749990479e-14 -0.06846776108008019 0
0 0 -0.1349407754720445
0 0 0.1349407754720445
0 0 0.1349407754720445
0 0 -0.1349407754720445
0 0 -0.1349407754720445
0 0 0.1349407754720445
0 0 0.1349407754720445
0 0 -0.1349407754720445
0 0 0.1349407754718785
0 0 -0.1349407754718785
0 0 -0.1349407754718785
0 0 0.1349407754718785
0 0 0.1349407754718785
0 0 -0.1349407754718785
0 0 -0.1349407754718785
0 0 0.1349407754718785
9.47102051738239e-14 0.1544579067183482 0
-9.47102051738239e-14 -0.1544579067183482 0
-9.47102051738239e-14 -0.1544579067183482 0
9.47102051738239e-14 0.1544579067183482 0
-9.47102051738239e-14 -0.1544579067183482 0
9.47102051738239e-14 0.1544579067183482 0
9.47102051738239e-14 0.1544579067183482 0
-9.47102051738239e-14 -0.1544579067183482 0
4.317295875443265e-14 0.07026637937184611 0
-4.317295875443265e-14 -0.07026637937184611 0
-4.317295875443265e-14 -0.07026637937184611 0
4.317295875443265e-14 0.07026637937184611 0

0 0 0
0 0 0
0 0 0
0 0 0
-0.339584297469156 -2.086213806299384e-13 0
0.339584297469156 2.086213806299384e-13 0
0.339584297469156 2.086213806299384e-13 0
-0.339584297469156 -2.086213806299384e-13 0
-0.06851526255698287 4.209089037723726e-14 0
0.06851526255698287 -4.209089037723726e-14 0
0.06851526255698287 -4.209089037723726e-14 0
-0.06851526255698287 4.209089037723726e-14 0
0.06851526255698287 -4.209089037723726e-14 0
-0.06851526255698287 4.209089037723726e-14 0
-0.06851526255698287 4.209089037723726e-14 0
0.06851526255698287 -4.209089037723726e-14 0
0 0 0.1349095608290206
0 0 -0.1349095608290206
0 0 -0.1349095608290206
0 0 0.1349095608290206
0 0 0.1349095608290206
0 0 -0.1349095608290206
0 0 -0.1349095608290206
0 0 0.1349095608290206
0 0 0.1349095608291865
0 0 -0.1349095608291865
0 0 -0.1349095608291865
0 0 0.1349095608291865
0 0 0.1349095608291865
0 0 -0.1349095608291865
0 0 -0.1349095608291865
0 0 0.1349095608291865
-0.1542070668833657 9.495141706783799e-14 0
0.1542070668833657 -9.495141706783799e-14 0
0.1542070668833657 -9.495141706783799e-14 0
-0.1542070668833657 9.495141706783799e-14 0
0.1542070668833657 -9.495141706783799e-14 0
-0.1542070668833657 9.495141706783799e-14 0
-0.1542070668833657 9.495141706783799e-14 0
0.1542070668833657 -9.495141706783799e-14 0
-0.07022695451126547 4.317509242929524e-14 0
0.07022695451126547 -4.317509242929524e-14 0
0.07022695451126547 -4.317509242929524e-14 0
-0.07022695451126547 4.317509242929524e-14 0

0 0 0
0 0 0
0 0 0
0 0 0
-6.795142608236205e-14 -0.1897966578295487 0
6.795142608236205e-14 0.1897966578295487 0
6.795142608236205e-14 0.1897966578295487 0
-6.795142608236205e-14 -0.1897966578295487 0
-3.190314195349643e-14 0.08889398851195889 0
3.190314195349643e-14 -0.08889398851195889 0
3.190314195349643e-14 -0.08889398851195889 0
-3.190314195349643e-14 0.08889398851195889 0
3.190314195349643e-14 -0.08889398851195889 0
-3.190314195349643e-14 0.08889398851195889 0
-3.190314195349643e-14 0.08889398851195889 0
3.190314195349643e-14 -0.08889398851195889 0
0 0 0.1482583681359469
0 0 -0.1482583681359469
0 0 -0.1482583681359469
0 0 0.1482583681359469
0 0 0.1482583681359469
0 0 -0.1482583681359469
0 0 -0.1482583681359469
0 0 0.1482583681359469
0 0 -0.1482583681360532
0 0 0.1482583681360532
0 0 0.1482583681360532
0 0 -0.1482583681360532
0 0 -0.1482583681360532
0 0 0.1482583681360532
0 0 0.1482583681360532
0 0 -0.1482583681360532
2.210800415693517e-14 -0.06170247799012821 0
-2.210800415693517e-14 0.06170247799012821 0
-2.210800415693517e-14 0.06170247799012821 0
2.210800415693517e-14 -0.06170247799012821 0
-2.210800415693517e-14 0.06170247799012821 0
2.210800415693517e-14 -0.06170247799012821 0
2.210800415693517e-14 -0.06170247799012821 0
-2.210800415693517e-14 0.06170247799012821 0
-1.147906873951064e-13 0.3203691305665415 0
1.147906873951064e-13 -0.3203691305665415 0
1.147906873951064e-13 -0.3203691305665415 0
-1.147906873951064e-13 0.3203691305665415 0

0 0 0
0 0 0
0 0 0
0 0 0
0.1895964131672402 -6.796800616031022e-14 0
-0.1895964131672402 6.796800616031022e-14 0
-0.1895964131672402 6.796800616031022e-14 0
0.1895964131672402 -6.796800616031022e-14 0
0.08888428516442287 3.184298694374334e-14 0
-0.08888428516442287 -3.184298694374334e-14 0
-0.08888428516442287 -3.184298694374334e-14 0
0.08888428516442287 3.184298694374334e-14 0
-0.08888428516442287 -3.184298694374334e-14 0
0.08888428516442287 3.184298694374334e-14 0
0.08888428516442287 3.184298694374334e-14 0
-0.08888428516442287 -3.184298694374334e-14 0
0 0 0.1483324455936597
0 0 -0.1483324455936597
0 0 -0.1483324455936597
0 0 0.1483324455936597
0 0 0.1483324455936597
0 0 -0.1483324455936597
0 0 -0.1483324455936597
0 0 0.1483324455936597
0 0 0.1483324455935535
0 0 -0.1483324455935535
0 0 -0.1483324455935535
0 0 0.1483324455935535
0 0 0.1483324455935535
0 0 -0.1483324455935535
0 0 -0.1483324455935535
0 0 0.1483324455935535
-0.06165975806832207 -2.205918852261643e-14 0
0.06165975806832207 2.205918852261643e-14 0
0.06165975806832207 2.205918852261643e-14 0
-0.06165975806832207 -2.205918852261643e-14 0
0.06165975806832207 2.205918852261643e-14 0
-0.06165975806832207 -2.205918852261643e-14 0
-0.06165975806832207 -2.205918852261643e-14 0
0.06165975806832207 2.205918852261643e-14 0
0.3203723749366155 1.14810981754218e-13 0
-0.3203723749366155 -1.14810981754218e-13 0
-0.3203723749366155 -1.14810981754218e-13 0
0.3203723749366155 1.14810981754218e-13 0

0 0 0
0 0 0
0 0 0
0 0 0
0.1363543708584901 -8.233948143471976e-17 0
-0.1363543708584901 8.233948143471976e-17 0
-0.1363543708584901 8.233948143471976e-17 0
0.1363543708584901 -8.233948143471976e-17 0
-0.3342798920242747 -6.461263327372558e-17 0
0.3342798920242747 6.461263327372558e-17 0
0.3342798920242747 6.461263327372558e-17 0
-0.3342798920242747 -6.461263327372558e-17 0
0.3342798920242747 6.461263327372558e-17 0
-0.3342798920242747 -6.461263327372558e-17 0
-0.3342798920242747 -6.461263327372558e-17 0
0.3342798920242747 6.461263327372558e-17 0
0 0 0.02183310908758256
0 0 -0.02183310908758256
0 0 -0.02183310908758256
0 0 0.02183310908758256
0 0 0.02183310908758256
0 0 -0.02183310908758256
0 0 -0.02183310908758256
0 0 0.02183310908758256
0 0 0.02183310908758259
0 0 -0.02183310908758259
0 0 -0.02183310908758259
0 0 0.02183310908758259
0 0 0.02183310908758259
0 0 -0.02183310908758259
0 0 -0.02183310908758259
0 0 0.02183310908758259
0.02016773510552705 3.629637279141869e-17 0
-0.02016773510552705 -3.629637279141869e-17 0
-0.02016773510552705 -3.629637279141869e-17 0
0.02016773510552705 3.629637279141869e-17 0
-0.02016773510552705 -3.629637279141869e-17 0
0.02016773510552705 3.629637279141869e-17 0
0.02016773510552705 3.629637279141869e-17 0
-0.02016773510552705 -3.629637279141869e-17 0
0.07211920296266267 5.336804490634663e-17 0
-0.07211920296266267 -5.336804490634663e-17 0
-0.07211920296266267 -5.336804490634663e-17 0
0.07211920296266267 5.336804490634663e-17 0

0 0 0
0 0 0
0 0 0
0 0 0
3.112963491223838e-17 0.1363550867523376 0
-3.112963491223838e-17 -0.1363550867523376 0
-3.112963491223838e-17 -0.1363550867523376 0
3.112963491223838e-17 0.1363550867523376 0
-6.604673230833534e-17 0.3342851237224897 0
6.604673230833534e-17 -0.3342851237224897 0
6.604673230833534e-17 -0.3342851237224897 0
-6.604673230833534e-17 0.3342851237224897 0
6.604673230833534e-17 -0.3342851237224897 0
-6.604673230833534e-17 0.3342851237224897 0
-6.604673230833534e-17 0.3342851237224897 0
6.604673230833534e-17 -0.3342851237224897 0
0 0 -0.02182801112201173
0 0 0.02182801112201173
0 0 0.02182801112201173
0 0 -0.02182801112201173
0 0 -0.02182801112201173
0 0 0.02182801112201173
0 0 0.02182801112201173
0 0 -0.02182801112201173
0 0 0.02182801112201175
0 0 -0.02182801112201175
0 0 -0.02182801112201175
0 0 0.02182801112201175
0 0 0.02182801112201175
0 0 -0.02182801112201175
0 0 -0.02182801112201175
0 0 0.02182801112201175
3.856723166539476e-18 -0.02014980698972823 0
-3.856723166539476e-18 0.02014980698972823 0
-3.856723166539476e-18 0.02014980698972823 0
3.856723166539476e-18 -0.02014980698972823 0
-3.856723166539476e-18 0.02014980698972823 0
3.856723166539476e-18 -0.02014980698972823 0
3.856723166539476e-18 -0.02014980698972823 0
-3.856723166539476e-18 0.02014980698972823 0
6.111691182861092e-17 -0.0720855374595004 0
-6.111691182861092e-17 0.0720855374595004 0
-6.111691182861092e-17 0.0720855374595004 0
6.111691182861092e-17 -0.0720855374595004 0                                                         
'''

def split_modes(input_data, columns_to_remove=None):
    data_arrays = []

    # Split the input data into individual arrays
    arrays = input_data.strip().split('\n\n')

    for array in arrays:
        # Splitting the data into rows and then into elements, converting each to a float
        array = [list(map(float, row.split())) for row in array.strip().split('\n')]
        # Converting the data to a NumPy matrix
        data_arrays.append(np.array(array))
    return data_arrays

A5minus_eigenvec = split_modes(A5minus_eigenvec)

In [8]:
for i in range(len(A5minus_eigenvec)):
    #moded_matrix = add_distortions_by_labels(np.array([row[:] for row in add_numbers2labels(labeled_matrix)]), np.array(result[i]['data']), lattice_basis) #labled matrix had columns of strings - convert to float    
    #    moded_matrix = add_distortions_by_labels(moded_matrix, np.array(result[i+1]['data']),lattice_basis)


    def matrix_to_string(matrix):
        # Update this function to handle labels correctly
        row_strings = [" ".join(str(value) for value in row) for row in matrix]
        return "\n".join(row_strings)
    
    #print(matrix_to_string(moded_matrix[:,1:]), '\n') 

    diff = np.array(I2cm-P4mmm_matrix)

    #convert to cart. coordinates
    cart_diff = np.dot(diff,np.linalg.inv(lattice_basis)).flatten()

    mode = np.array(A5minus_eigenvec[i].flatten())
    #print(np.dot(mode,mode))
    print(np.dot(mode, cart_diff))

-0.01969682285658097
1.0117774673634727e-15
1.4992185010803682e-14
0.10704007969265251
3.161674481250154e-14
-0.0508131805687899
-4.609268695887625e-14
0.12749509889872926
-0.09361106452243627
4.347650711666873e-17


In [9]:
np.linalg.norm([-0.01969682285658097,
    1.0117774673634727e-15,
    1.4992185010803682e-14,
    0.10704007969265251,
    3.161674481250154e-14,
    -0.0508131805687899,
    -4.609268695887625e-14,
    0.12749509889872926,
    -0.09361106452243627,
    4.347650711666873e-17])

0.19860904927769493

In [28]:
np.linalg.norm(np.array(A5minus_dist-P4mmm_dist)) - np.linalg.norm([-0.5889870556341774,-0.19931552718578013,-0.07945481209040099,0.16963292467914712,-0.2726484457272456,0.6671695393753513,-0.33237781136607536,-0.005488815825143878,0.5501368302318954,0.8770108716449216])

1.6449708917907233

In [14]:
GM5m_eigenvec_super = '''
-4.282264356115649e-13 -0.01257767725184052 0
-4.282264356115649e-13 -0.01257767725184052 0
-4.282264356115649e-13 -0.01257767725184052 0
-4.282264356115649e-13 -0.01257767725184052 0
-1.384181624982011e-11 -0.4069754990069022 0
-1.384181624982011e-11 -0.4069754990069022 0
-1.384181624982011e-11 -0.4069754990069022 0
-1.384181624982011e-11 -0.4069754990069022 0
-1.325659925579004e-12 -0.03897550780621601 0
-1.325659925579004e-12 -0.03897550780621601 0
-1.325659925579004e-12 -0.03897550780621601 0
-1.325659925579004e-12 -0.03897550780621601 0
-1.325659925579004e-12 -0.03897550780621601 0
-1.325659925579004e-12 -0.03897550780621601 0
-1.325659925579004e-12 -0.03897550780621601 0
-1.325659925579004e-12 -0.03897550780621601 0
-0.01096309476322607 0.03790457904264066 0
-0.01096309476322607 0.03790457904264066 0
-0.01096309476322607 0.03790457904264066 0
-0.01096309476322607 0.03790457904264066 0
-0.01096309476322607 0.03790457904264066 0
-0.01096309476322607 0.03790457904264066 0
-0.01096309476322607 0.03790457904264066 0
-0.01096309476322607 0.03790457904264066 0
0.01096309476580403 0.03790457904338629 0
0.01096309476580403 0.03790457904338629 0
0.01096309476580403 0.03790457904338629 0
0.01096309476580403 0.03790457904338629 0
0.01096309476580403 0.03790457904338629 0
0.01096309476580403 0.03790457904338629 0
0.01096309476580403 0.03790457904338629 0
0.01096309476580403 0.03790457904338629 0
1.336972852490332e-12 0.03931571295173966 0
1.336972852490332e-12 0.03931571295173966 0
1.336972852490332e-12 0.03931571295173966 0
1.336972852490332e-12 0.03931571295173966 0
1.336972852490332e-12 0.03931571295173966 0
1.336972852490332e-12 0.03931571295173966 0
1.336972852490332e-12 0.03931571295173966 0
1.336972852490332e-12 0.03931571295173966 0
9.116876177635958e-12 0.26805844142415 0
9.116876177635958e-12 0.26805844142415 0
9.116876177635958e-12 0.26805844142415 0
9.116876177635958e-12 0.26805844142415 0

-0.01258253228503869 4.278018195791409e-13 0
-0.01258253228503869 4.278018195791409e-13 0
-0.01258253228503869 4.278018195791409e-13 0
-0.01258253228503869 4.278018195791409e-13 0
-0.4069780415370147 1.384180385641756e-11 0
-0.4069780415370147 1.384180385641756e-11 0
-0.4069780415370147 1.384180385641756e-11 0
-0.4069780415370147 1.384180385641756e-11 0
-0.03897639691403009 1.325632954128114e-12 0
-0.03897639691403009 1.325632954128114e-12 0
-0.03897639691403009 1.325632954128114e-12 0
-0.03897639691403009 1.325632954128114e-12 0
-0.03897639691403009 1.325632954128114e-12 0
-0.03897639691403009 1.325632954128114e-12 0
-0.03897639691403009 1.325632954128114e-12 0
-0.03897639691403009 1.325632954128114e-12 0
0.03790156260032922 -0.01096178493693309 0
0.03790156260032922 -0.01096178493693309 0
0.03790156260032922 -0.01096178493693309 0
0.03790156260032922 -0.01096178493693309 0
0.03790156260032922 -0.01096178493693309 0
0.03790156260032922 -0.01096178493693309 0
0.03790156260032922 -0.01096178493693309 0
0.03790156260032922 -0.01096178493693309 0
0.03790156259958349 0.01096178493435483 0
0.03790156259958349 0.01096178493435483 0
0.03790156259958349 0.01096178493435483 0
0.03790156259958349 0.01096178493435483 0
0.03790156259958349 0.01096178493435483 0
0.03790156259958349 0.01096178493435483 0
0.03790156259958349 0.01096178493435483 0
0.03790156259958349 0.01096178493435483 0
0.03930925529971355 -1.337097616221614e-12 0
0.03930925529971355 -1.337097616221614e-12 0
0.03930925529971355 -1.337097616221614e-12 0
0.03930925529971355 -1.337097616221614e-12 0
0.03930925529971355 -1.337097616221614e-12 0
0.03930925529971355 -1.337097616221614e-12 0
0.03930925529971355 -1.337097616221614e-12 0
0.03930925529971355 -1.337097616221614e-12 0
0.2680579093121124 -9.116814072465535e-12 0
0.2680579093121124 -9.116814072465535e-12 0
0.2680579093121124 -9.116814072465535e-12 0
0.2680579093121124 -9.116814072465535e-12 0

6.197828930954534e-12 -0.1501302831962604 0
6.197828930954534e-12 -0.1501302831962604 0
6.197828930954534e-12 -0.1501302831962604 0
6.197828930954534e-12 -0.1501302831962604 0
6.233851355368126e-12 -0.1510022885447787 0
6.233851355368126e-12 -0.1510022885447787 0
6.233851355368126e-12 -0.1510022885447787 0
6.233851355368126e-12 -0.1510022885447787 0
6.224250948564762e-12 -0.1507536164980965 0
6.224250948564762e-12 -0.1507536164980965 0
6.224250948564762e-12 -0.1507536164980965 0
6.224250948564762e-12 -0.1507536164980965 0
6.224250948564762e-12 -0.1507536164980965 0
6.224250948564762e-12 -0.1507536164980965 0
6.224250948564762e-12 -0.1507536164980965 0
6.224250948564762e-12 -0.1507536164980965 0
0.0001284826806262394 -0.150940234235623 0
0.0001284826806262394 -0.150940234235623 0
0.0001284826806262394 -0.150940234235623 0
0.0001284826806262394 -0.150940234235623 0
0.0001284826806262394 -0.150940234235623 0
0.0001284826806262394 -0.150940234235623 0
0.0001284826806262394 -0.150940234235623 0
0.0001284826806262394 -0.150940234235623 0
-0.0001284826681624231 -0.1509402342356125 0
-0.0001284826681624231 -0.1509402342356125 0
-0.0001284826681624231 -0.1509402342356125 0
-0.0001284826681624231 -0.1509402342356125 0
-0.0001284826681624231 -0.1509402342356125 0
-0.0001284826681624231 -0.1509402342356125 0
-0.0001284826681624231 -0.1509402342356125 0
-0.0001284826681624231 -0.1509402342356125 0
6.220502483875479e-12 -0.1506826510759754 0
6.220502483875479e-12 -0.1506826510759754 0
6.220502483875479e-12 -0.1506826510759754 0
6.220502483875479e-12 -0.1506826510759754 0
6.220502483875479e-12 -0.1506826510759754 0
6.220502483875479e-12 -0.1506826510759754 0
6.220502483875479e-12 -0.1506826510759754 0
6.220502483875479e-12 -0.1506826510759754 0
6.215970120168267e-12 -0.1505439975587485 0
6.215970120168267e-12 -0.1505439975587485 0
6.215970120168267e-12 -0.1505439975587485 0
6.215970120168267e-12 -0.1505439975587485 0

-0.1500751555436455 -6.198809191554106e-12 0
-0.1500751555436455 -6.198809191554106e-12 0
-0.1500751555436455 -6.198809191554106e-12 0
-0.1500751555436455 -6.198809191554106e-12 0
-0.1510015849570343 -6.234025353700659e-12 0
-0.1510015849570343 -6.234025353700659e-12 0
-0.1510015849570343 -6.234025353700659e-12 0
-0.1510015849570343 -6.234025353700659e-12 0
-0.1507640335702539 -6.223965824398054e-12 0
-0.1507640335702539 -6.223965824398054e-12 0
-0.1507640335702539 -6.223965824398054e-12 0
-0.1507640335702539 -6.223965824398054e-12 0
-0.1507640335702539 -6.223965824398054e-12 0
-0.1507640335702539 -6.223965824398054e-12 0
-0.1507640335702539 -6.223965824398054e-12 0
-0.1507640335702539 -6.223965824398054e-12 0
-0.1509471658743203 0.000125689794377673 0
-0.1509471658743203 0.000125689794377673 0
-0.1509471658743203 0.000125689794377673 0
-0.1509471658743203 0.000125689794377673 0
-0.1509471658743203 0.000125689794377673 0
-0.1509471658743203 0.000125689794377673 0
-0.1509471658743203 0.000125689794377673 0
-0.1509471658743203 0.000125689794377673 0
-0.1509471658743309 -0.0001256898068408424 0
-0.1509471658743309 -0.0001256898068408424 0
-0.1509471658743309 -0.0001256898068408424 0
-0.1509471658743309 -0.0001256898068408424 0
-0.1509471658743309 -0.0001256898068408424 0
-0.1509471658743309 -0.0001256898068408424 0
-0.1509471658743309 -0.0001256898068408424 0
-0.1509471658743309 -0.0001256898068408424 0
-0.1506777341541853 -6.221144500065569e-12 0
-0.1506777341541853 -6.221144500065569e-12 0
-0.1506777341541853 -6.221144500065569e-12 0
-0.1506777341541853 -6.221144500065569e-12 0
-0.1506777341541853 -6.221144500065569e-12 0
-0.1506777341541853 -6.221144500065569e-12 0
-0.1506777341541853 -6.221144500065569e-12 0
-0.1506777341541853 -6.221144500065569e-12 0
-0.1505608565295826 -6.215421301294508e-12 0
-0.1505608565295826 -6.215421301294508e-12 0
-0.1505608565295826 -6.215421301294508e-12 0
-0.1505608565295826 -6.215421301294508e-12 0

-0.4720304586033233 -2.324937037311279e-11 0
-0.4720304586033233 -2.324937037311279e-11 0
-0.4720304586033233 -2.324937037311279e-11 0
-0.4720304586033233 -2.324937037311279e-11 0
0.0570041849591898 2.808027248267549e-12 0
0.0570041849591898 2.808027248267549e-12 0
0.0570041849591898 2.808027248267549e-12 0
0.0570041849591898 2.808027248267549e-12 0
0.03136080895067487 1.544683675981957e-12 0
0.03136080895067487 1.544683675981957e-12 0
0.03136080895067487 1.544683675981957e-12 0
0.03136080895067487 1.544683675981957e-12 0
0.03136080895067487 1.544683675981957e-12 0
0.03136080895067487 1.544683675981957e-12 0
0.03136080895067487 1.544683675981957e-12 0
0.03136080895067487 1.544683675981957e-12 0
0.06710637486498516 -0.02083884632164039 0
0.06710637486498516 -0.02083884632164039 0
0.06710637486498516 -0.02083884632164039 0
0.06710637486498516 -0.02083884632164039 0
0.06710637486498516 -0.02083884632164039 0
0.06710637486498516 -0.02083884632164039 0
0.06710637486498516 -0.02083884632164039 0
0.06710637486498516 -0.02083884632164039 0
0.06710637486703976 0.02083884632825378 0
0.06710637486703976 0.02083884632825378 0
0.06710637486703976 0.02083884632825378 0
0.06710637486703976 0.02083884632825378 0
0.06710637486703976 0.02083884632825378 0
0.06710637486703976 0.02083884632825378 0
0.06710637486703976 0.02083884632825378 0
0.06710637486703976 0.02083884632825378 0
0.02880083718979721 1.417103842888832e-12 0
0.02880083718979721 1.417103842888832e-12 0
0.02880083718979721 1.417103842888832e-12 0
0.02880083718979721 1.417103842888832e-12 0
0.02880083718979721 1.417103842888832e-12 0
0.02880083718979721 1.417103842888832e-12 0
0.02880083718979721 1.417103842888832e-12 0
0.02880083718979721 1.417103842888832e-12 0
0.02370020402606066 1.167052988396322e-12 0
0.02370020402606066 1.167052988396322e-12 0
0.02370020402606066 1.167052988396322e-12 0
0.02370020402606066 1.167052988396322e-12 0

-0.4720304586033233 -2.324937037311279e-11 0
-0.4720304586033233 -2.324937037311279e-11 0
-0.4720304586033233 -2.324937037311279e-11 0
-0.4720304586033233 -2.324937037311279e-11 0
0.0570041849591898 2.808027248267549e-12 0
0.0570041849591898 2.808027248267549e-12 0
0.0570041849591898 2.808027248267549e-12 0
0.0570041849591898 2.808027248267549e-12 0
0.03136080895067487 1.544683675981957e-12 0
0.03136080895067487 1.544683675981957e-12 0
0.03136080895067487 1.544683675981957e-12 0
0.03136080895067487 1.544683675981957e-12 0
0.03136080895067487 1.544683675981957e-12 0
0.03136080895067487 1.544683675981957e-12 0
0.03136080895067487 1.544683675981957e-12 0
0.03136080895067487 1.544683675981957e-12 0
0.06710637486498516 -0.02083884632164039 0
0.06710637486498516 -0.02083884632164039 0
0.06710637486498516 -0.02083884632164039 0
0.06710637486498516 -0.02083884632164039 0
0.06710637486498516 -0.02083884632164039 0
0.06710637486498516 -0.02083884632164039 0
0.06710637486498516 -0.02083884632164039 0
0.06710637486498516 -0.02083884632164039 0
0.06710637486703976 0.02083884632825378 0
0.06710637486703976 0.02083884632825378 0
0.06710637486703976 0.02083884632825378 0
0.06710637486703976 0.02083884632825378 0
0.06710637486703976 0.02083884632825378 0
0.06710637486703976 0.02083884632825378 0
0.06710637486703976 0.02083884632825378 0
0.06710637486703976 0.02083884632825378 0
0.02880083718979721 1.417103842888832e-12 0
0.02880083718979721 1.417103842888832e-12 0
0.02880083718979721 1.417103842888832e-12 0
0.02880083718979721 1.417103842888832e-12 0
0.02880083718979721 1.417103842888832e-12 0
0.02880083718979721 1.417103842888832e-12 0
0.02880083718979721 1.417103842888832e-12 0
0.02880083718979721 1.417103842888832e-12 0
0.02370020402606066 1.167052988396322e-12 0
0.02370020402606066 1.167052988396322e-12 0
0.02370020402606066 1.167052988396322e-12 0
0.02370020402606066 1.167052988396322e-12 0

-2.325025788536945e-11 0.4720049749764643 0
-2.325025788536945e-11 0.4720049749764643 0
-2.325025788536945e-11 0.4720049749764643 0
-2.325025788536945e-11 0.4720049749764643 0
2.80755189678069e-12 -0.05701737470140049 0
2.80755189678069e-12 -0.05701737470140049 0
2.80755189678069e-12 -0.05701737470140049 0
2.80755189678069e-12 -0.05701737470140049 0
1.544487417823785e-12 -0.03137164207409616 0
1.544487417823785e-12 -0.03137164207409616 0
1.544487417823785e-12 -0.03137164207409616 0
1.544487417823785e-12 -0.03137164207409616 0
1.544487417823785e-12 -0.03137164207409616 0
1.544487417823785e-12 -0.03137164207409616 0
1.544487417823785e-12 -0.03137164207409616 0
1.544487417823785e-12 -0.03137164207409616 0
0.02085523133048732 -0.06714284688570177 0
0.02085523133048732 -0.06714284688570177 0
0.02085523133048732 -0.06714284688570177 0
0.02085523133048732 -0.06714284688570177 0
0.02085523133048732 -0.06714284688570177 0
0.02085523133048732 -0.06714284688570177 0
0.02085523133048732 -0.06714284688570177 0
0.02085523133048732 -0.06714284688570177 0
-0.02085523132387682 -0.06714284688364879 0
-0.02085523132387682 -0.06714284688364879 0
-0.02085523132387682 -0.06714284688364879 0
-0.02085523132387682 -0.06714284688364879 0
-0.02085523132387682 -0.06714284688364879 0
-0.02085523132387682 -0.06714284688364879 0
-0.02085523132387682 -0.06714284688364879 0
-0.02085523132387682 -0.06714284688364879 0
1.418312193817529e-12 -0.02878786953751064 0
1.418312193817529e-12 -0.02878786953751064 0
1.418312193817529e-12 -0.02878786953751064 0
1.418312193817529e-12 -0.02878786953751064 0
1.418312193817529e-12 -0.02878786953751064 0
1.418312193817529e-12 -0.02878786953751064 0
1.418312193817529e-12 -0.02878786953751064 0
1.418312193817529e-12 -0.02878786953751064 0
1.167160645422536e-12 -0.02370800741949137 0
1.167160645422536e-12 -0.02370800741949137 0
1.167160645422536e-12 -0.02370800741949137 0
1.167160645422536e-12 -0.02370800741949137 0

-8.694821910919097e-14 0.06407334289547935 0
-8.694821910919097e-14 0.06407334289547935 0
-8.694821910919097e-14 0.06407334289547935 0
-8.694821910919097e-14 0.06407334289547935 0
4.112249540085176e-14 -0.03009403852094789 0
4.112249540085176e-14 -0.03009403852094789 0
4.112249540085176e-14 -0.03009403852094789 0
4.112249540085176e-14 -0.03009403852094789 0
7.465761558674053e-14 -0.05461199351338741 0
7.465761558674053e-14 -0.05461199351338741 0
7.465761558674053e-14 -0.05461199351338741 0
7.465761558674053e-14 -0.05461199351338741 0
7.465761558674053e-14 -0.05461199351338741 0
7.465761558674053e-14 -0.05461199351338741 0
7.465761558674053e-14 -0.05461199351338741 0
7.465761558674053e-14 -0.05461199351338741 0
-0.1263081702756432 0.1446865503996617 0
-0.1263081702756432 0.1446865503996617 0
-0.1263081702756432 0.1446865503996617 0
-0.1263081702756432 0.1446865503996617 0
-0.1263081702756432 0.1446865503996617 0
-0.1263081702756432 0.1446865503996617 0
-0.1263081702756432 0.1446865503996617 0
-0.1263081702756432 0.1446865503996617 0
0.1263081702752479 0.1446865503993165 0
0.1263081702752479 0.1446865503993165 0
0.1263081702752479 0.1446865503993165 0
0.1263081702752479 0.1446865503993165 0
0.1263081702752479 0.1446865503993165 0
0.1263081702752479 0.1446865503993165 0
0.1263081702752479 0.1446865503993165 0
0.1263081702752479 0.1446865503993165 0
2.750153603502679e-13 -0.2013289383199808 0
2.750153603502679e-13 -0.2013289383199808 0
2.750153603502679e-13 -0.2013289383199808 0
2.750153603502679e-13 -0.2013289383199808 0
2.750153603502679e-13 -0.2013289383199808 0
2.750153603502679e-13 -0.2013289383199808 0
2.750153603502679e-13 -0.2013289383199808 0
2.750153603502679e-13 -0.2013289383199808 0
1.394425874593487e-13 -0.102007480682767 0
1.394425874593487e-13 -0.102007480682767 0
1.394425874593487e-13 -0.102007480682767 0
1.394425874593487e-13 -0.102007480682767 0

0.06401083598067496 8.746535399652366e-14 0
0.06401083598067496 8.746535399652366e-14 0
0.06401083598067496 8.746535399652366e-14 0
0.06401083598067496 8.746535399652366e-14 0
-0.03008557872319828 -4.10467037007685e-14 0
-0.03008557872319828 -4.10467037007685e-14 0
-0.03008557872319828 -4.10467037007685e-14 0
-0.03008557872319828 -4.10467037007685e-14 0
-0.05461079952991615 -7.451660519232148e-14 0
-0.05461079952991615 -7.451660519232148e-14 0
-0.05461079952991615 -7.451660519232148e-14 0
-0.05461079952991615 -7.451660519232148e-14 0
-0.05461079952991615 -7.451660519232148e-14 0
-0.05461079952991615 -7.451660519232148e-14 0
-0.05461079952991615 -7.451660519232148e-14 0
-0.05461079952991615 -7.451660519232148e-14 0
0.1446976339005633 -0.12631160386148 0
0.1446976339005633 -0.12631160386148 0
0.1446976339005633 -0.12631160386148 0
0.1446976339005633 -0.12631160386148 0
0.1446976339005633 -0.12631160386148 0
0.1446976339005633 -0.12631160386148 0
0.1446976339005633 -0.12631160386148 0
0.1446976339005633 -0.12631160386148 0
0.1446976339009085 0.1263116038618758 0
0.1446976339009085 0.1263116038618758 0
0.1446976339009085 0.1263116038618758 0
0.1446976339009085 0.1263116038618758 0
0.1446976339009085 0.1263116038618758 0
0.1446976339009085 0.1263116038618758 0
0.1446976339009085 0.1263116038618758 0
0.1446976339009085 0.1263116038618758 0
-0.2013196725974192 -2.747043508428644e-13 0
-0.2013196725974192 -2.747043508428644e-13 0
-0.2013196725974192 -2.747043508428644e-13 0
-0.2013196725974192 -2.747043508428644e-13 0
-0.2013196725974192 -2.747043508428644e-13 0
-0.2013196725974192 -2.747043508428644e-13 0
-0.2013196725974192 -2.747043508428644e-13 0
-0.2013196725974192 -2.747043508428644e-13 0
-0.1020071797647829 -1.391619140505205e-13 0
-0.1020071797647829 -1.391619140505205e-13 0
-0.1020071797647829 -1.391619140505205e-13 0
-0.1020071797647829 -1.391619140505205e-13 0

2.036297053006807e-14 -0.01742953081743166 0
2.036297053006807e-14 -0.01742953081743166 0
2.036297053006807e-14 -0.01742953081743166 0
2.036297053006807e-14 -0.01742953081743166 0
-8.290797954425287e-14 0.07212402998868259 0
-8.290797954425287e-14 0.07212402998868259 0
-8.290797954425287e-14 0.07212402998868259 0
-8.290797954425287e-14 0.07212402998868259 0
-1.490327678312259e-13 0.129695615832934 0
-1.490327678312259e-13 0.129695615832934 0
-1.490327678312259e-13 0.129695615832934 0
-1.490327678312259e-13 0.129695615832934 0
-1.490327678312259e-13 0.129695615832934 0
-1.490327678312259e-13 0.129695615832934 0
-1.490327678312259e-13 0.129695615832934 0
-1.490327678312259e-13 0.129695615832934 0
0.09031888647669921 -0.0174557434053386 0
0.09031888647669921 -0.0174557434053386 0
0.09031888647669921 -0.0174557434053386 0
0.09031888647669921 -0.0174557434053386 0
0.09031888647669921 -0.0174557434053386 0
0.09031888647669921 -0.0174557434053386 0
0.09031888647669921 -0.0174557434053386 0
0.09031888647669921 -0.0174557434053386 0
-0.09031888647665896 -0.01745574340513087 0
-0.09031888647665896 -0.01745574340513087 0
-0.09031888647665896 -0.01745574340513087 0
-0.09031888647665896 -0.01745574340513087 0
-0.09031888647665896 -0.01745574340513087 0
-0.09031888647665896 -0.01745574340513087 0
-0.09031888647665896 -0.01745574340513087 0
-0.09031888647665896 -0.01745574340513087 0
2.793984418301644e-13 -0.2431638567237038 0
2.793984418301644e-13 -0.2431638567237038 0
2.793984418301644e-13 -0.2431638567237038 0
2.793984418301644e-13 -0.2431638567237038 0
2.793984418301644e-13 -0.2431638567237038 0
2.793984418301644e-13 -0.2431638567237038 0
2.793984418301644e-13 -0.2431638567237038 0
2.793984418301644e-13 -0.2431638567237038 0
-2.786356752975767e-13 0.2423763413758636 0
-2.786356752975767e-13 0.2423763413758636 0
-2.786356752975767e-13 0.2423763413758636 0
-2.786356752975767e-13 0.2423763413758636 0

-0.01744013971578567 -2.003563603668363e-14 0
-0.01744013971578567 -2.003563603668363e-14 0
-0.01744013971578567 -2.003563603668363e-14 0
-0.01744013971578567 -2.003563603668363e-14 0
0.07212807986541991 8.286893630242219e-14 0
0.07212807986541991 8.286893630242219e-14 0
0.07212807986541991 8.286893630242219e-14 0
0.07212807986541991 8.286893630242219e-14 0
0.1296843107181705 1.490982891748591e-13 0
0.1296843107181705 1.490982891748591e-13 0
0.1296843107181705 1.490982891748591e-13 0
0.1296843107181705 1.490982891748591e-13 0
0.1296843107181705 1.490982891748591e-13 0
0.1296843107181705 1.490982891748591e-13 0
0.1296843107181705 1.490982891748591e-13 0
0.1296843107181705 1.490982891748591e-13 0
-0.01745973016969627 0.09031539294722046 0
-0.01745973016969627 0.09031539294722046 0
-0.01745973016969627 0.09031539294722046 0
-0.01745973016969627 0.09031539294722046 0
-0.01745973016969627 0.09031539294722046 0
-0.01745973016969627 0.09031539294722046 0
-0.01745973016969627 0.09031539294722046 0
-0.01745973016969627 0.09031539294722046 0
-0.01745973016990391 -0.09031539294726054 0
-0.01745973016990391 -0.09031539294726054 0
-0.01745973016990391 -0.09031539294726054 0
-0.01745973016990391 -0.09031539294726054 0
-0.01745973016990391 -0.09031539294726054 0
-0.01745973016990391 -0.09031539294726054 0
-0.01745973016990391 -0.09031539294726054 0
-0.01745973016990391 -0.09031539294726054 0
-0.2431737265686809 -2.794500769023677e-13 0
-0.2431737265686809 -2.794500769023677e-13 0
-0.2431737265686809 -2.794500769023677e-13 0
-0.2431737265686809 -2.794500769023677e-13 0
-0.2431737265686809 -2.794500769023677e-13 0
-0.2431737265686809 -2.794500769023677e-13 0
-0.2431737265686809 -2.794500769023677e-13 0
-0.2431737265686809 -2.794500769023677e-13 0
0.2423707255814141 2.786127931708126e-13 0
0.2423707255814141 2.786127931708126e-13 0
0.2423707255814141 2.786127931708126e-13 0
0.2423707255814141 2.786127931708126e-13 0

-3.468566383352492e-13 0.01019990029939194 0
-3.468566383352492e-13 0.01019990029939194 0
-3.468566383352492e-13 0.01019990029939194 0
-3.468566383352492e-13 0.01019990029939194 0
-7.746682233032632e-12 0.2278562951903609 0
-7.746682233032632e-12 0.2278562951903609 0
-7.746682233032632e-12 0.2278562951903609 0
-7.746682233032632e-12 0.2278562951903609 0
8.064116843679208e-12 -0.2371884334980031 0
8.064116843679208e-12 -0.2371884334980031 0
8.064116843679208e-12 -0.2371884334980031 0
8.064116843679208e-12 -0.2371884334980031 0
8.064116843679208e-12 -0.2371884334980031 0
8.064116843679208e-12 -0.2371884334980031 0
8.064116843679208e-12 -0.2371884334980031 0
8.064116843679208e-12 -0.2371884334980031 0
-0.04535732421609745 -0.0161832361451492 0
-0.04535732421609745 -0.0161832361451492 0
-0.04535732421609745 -0.0161832361451492 0
-0.04535732421609745 -0.0161832361451492 0
-0.04535732421609745 -0.0161832361451492 0
-0.04535732421609745 -0.0161832361451492 0
-0.04535732421609745 -0.0161832361451492 0
-0.04535732421609745 -0.0161832361451492 0
0.04535732421719774 -0.01618323614823351 0
0.04535732421719774 -0.01618323614823351 0
0.04535732421719774 -0.01618323614823351 0
0.04535732421719774 -0.01618323614823351 0
0.04535732421719774 -0.01618323614823351 0
0.04535732421719774 -0.01618323614823351 0
0.04535732421719774 -0.01618323614823351 0
0.04535732421719774 -0.01618323614823351 0
-4.358012406324715e-13 0.01281056777141826 0
-4.358012406324715e-13 0.01281056777141826 0
-4.358012406324715e-13 0.01281056777141826 0
-4.358012406324715e-13 0.01281056777141826 0
-4.358012406324715e-13 0.01281056777141826 0
-4.358012406324715e-13 0.01281056777141826 0
-4.358012406324715e-13 0.01281056777141826 0
-4.358012406324715e-13 0.01281056777141826 0
-9.363751818012011e-12 0.2754193303731337 0
-9.363751818012011e-12 0.2754193303731337 0
-9.363751818012011e-12 0.2754193303731337 0
-9.363751818012011e-12 0.2754193303731337 0

-0.01020021148126352 -3.467799134283376e-13 0
-0.01020021148126352 -3.467799134283376e-13 0
-0.01020021148126352 -3.467799134283376e-13 0
-0.01020021148126352 -3.467799134283376e-13 0
-0.2278553545690039 -7.746758873873058e-12 0
-0.2278553545690039 -7.746758873873058e-12 0
-0.2278553545690039 -7.746758873873058e-12 0
-0.2278553545690039 -7.746758873873058e-12 0
0.237189862176068 8.064038636591634e-12 0
0.237189862176068 8.064038636591634e-12 0
0.237189862176068 8.064038636591634e-12 0
0.237189862176068 8.064038636591634e-12 0
0.237189862176068 8.064038636591634e-12 0
0.237189862176068 8.064038636591634e-12 0
0.237189862176068 8.064038636591634e-12 0
0.237189862176068 8.064038636591634e-12 0
0.01618180458988307 0.04535863317922682 0
0.01618180458988307 0.04535863317922682 0
0.01618180458988307 0.04535863317922682 0
0.01618180458988307 0.04535863317922682 0
0.01618180458988307 0.04535863317922682 0
0.01618180458988307 0.04535863317922682 0
0.01618180458988307 0.04535863317922682 0
0.01618180458988307 0.04535863317922682 0
0.01618180458679894 -0.04535863317812641 0
0.01618180458679894 -0.04535863317812641 0
0.01618180458679894 -0.04535863317812641 0
0.01618180458679894 -0.04535863317812641 0
0.01618180458679894 -0.04535863317812641 0
0.01618180458679894 -0.04535863317812641 0
0.01618180458679894 -0.04535863317812641 0
0.01618180458679894 -0.04535863317812641 0
-0.01281786795407119 -4.355298919866317e-13 0
-0.01281786795407119 -4.355298919866317e-13 0
-0.01281786795407119 -4.355298919866317e-13 0
-0.01281786795407119 -4.355298919866317e-13 0
-0.01281786795407119 -4.355298919866317e-13 0
-0.01281786795407119 -4.355298919866317e-13 0
-0.01281786795407119 -4.355298919866317e-13 0
-0.01281786795407119 -4.355298919866317e-13 0
-0.2754164311541681 -9.36385904341725e-12 0
-0.2754164311541681 -9.36385904341725e-12 0
-0.2754164311541681 -9.36385904341725e-12 0
-0.2754164311541681 -9.36385904341725e-12 0

1.339808569353209e-16 0.0008853922246728528 0
1.339808569353209e-16 0.0008853922246728528 0
1.339808569353209e-16 0.0008853922246728528 0
1.339808569353209e-16 0.0008853922246728528 0
-3.072285785716104e-18 -0.01709540670171136 0
-3.072285785716104e-18 -0.01709540670171136 0
-3.072285785716104e-18 -0.01709540670171136 0
-3.072285785716104e-18 -0.01709540670171136 0
-4.897013469559062e-17 -0.1539750024859241 0
-4.897013469559062e-17 -0.1539750024859241 0
-4.897013469559062e-17 -0.1539750024859241 0
-4.897013469559062e-17 -0.1539750024859241 0
-4.897013469559062e-17 -0.1539750024859241 0
-4.897013469559062e-17 -0.1539750024859241 0
-4.897013469559062e-17 -0.1539750024859241 0
-4.897013469559062e-17 -0.1539750024859241 0
0.1891462697989682 0.1107760295017699 0
0.1891462697989682 0.1107760295017699 0
0.1891462697989682 0.1107760295017699 0
0.1891462697989682 0.1107760295017699 0
0.1891462697989682 0.1107760295017699 0
0.1891462697989682 0.1107760295017699 0
0.1891462697989682 0.1107760295017699 0
0.1891462697989682 0.1107760295017699 0
-0.1891462697989681 0.1107760295017697 0
-0.1891462697989681 0.1107760295017697 0
-0.1891462697989681 0.1107760295017697 0
-0.1891462697989681 0.1107760295017697 0
-0.1891462697989681 0.1107760295017697 0
-0.1891462697989681 0.1107760295017697 0
-0.1891462697989681 0.1107760295017697 0
-0.1891462697989681 0.1107760295017697 0
3.243956762934019e-17 -0.009703539502720758 0
3.243956762934019e-17 -0.009703539502720758 0
3.243956762934019e-17 -0.009703539502720758 0
3.243956762934019e-17 -0.009703539502720758 0
3.243956762934019e-17 -0.009703539502720758 0
3.243956762934019e-17 -0.009703539502720758 0
3.243956762934019e-17 -0.009703539502720758 0
3.243956762934019e-17 -0.009703539502720758 0
-2.449503574595989e-17 -0.09955642946241472 0
-2.449503574595989e-17 -0.09955642946241472 0
-2.449503574595989e-17 -0.09955642946241472 0
-2.449503574595989e-17 -0.09955642946241472 0

-0.0008853304860709655 3.095927245120619e-19 0
-0.0008853304860709655 3.095927245120619e-19 0
-0.0008853304860709655 3.095927245120619e-19 0
-0.0008853304860709655 3.095927245120619e-19 0
0.01709542004015118 -1.067399824374762e-17 0
0.01709542004015118 -1.067399824374762e-17 0
0.01709542004015118 -1.067399824374762e-17 0
0.01709542004015118 -1.067399824374762e-17 0
0.1539745295051774 -5.875631749513119e-17 0
0.1539745295051774 -5.875631749513119e-17 0
0.1539745295051774 -5.875631749513119e-17 0
0.1539745295051774 -5.875631749513119e-17 0
0.1539745295051774 -5.875631749513119e-17 0
0.1539745295051774 -5.875631749513119e-17 0
0.1539745295051774 -5.875631749513119e-17 0
0.1539745295051774 -5.875631749513119e-17 0
-0.1107748206296166 -0.1891472148348536 0
-0.1107748206296166 -0.1891472148348536 0
-0.1107748206296166 -0.1891472148348536 0
-0.1107748206296166 -0.1891472148348536 0
-0.1107748206296166 -0.1891472148348536 0
-0.1107748206296166 -0.1891472148348536 0
-0.1107748206296166 -0.1891472148348536 0
-0.1107748206296166 -0.1891472148348536 0
-0.1107748206296168 0.1891472148348537 0
-0.1107748206296168 0.1891472148348537 0
-0.1107748206296168 0.1891472148348537 0
-0.1107748206296168 0.1891472148348537 0
-0.1107748206296168 0.1891472148348537 0
-0.1107748206296168 0.1891472148348537 0
-0.1107748206296168 0.1891472148348537 0
-0.1107748206296168 0.1891472148348537 0
0.009702837120640058 -3.524468478764323e-18 0
0.009702837120640058 -3.524468478764323e-18 0
0.009702837120640058 -3.524468478764323e-18 0
0.009702837120640058 -3.524468478764323e-18 0
0.009702837120640058 -3.524468478764323e-18 0
0.009702837120640058 -3.524468478764323e-18 0
0.009702837120640058 -3.524468478764323e-18 0
0.009702837120640058 -3.524468478764323e-18 0
0.09955622619051102 -6.429046385479891e-17 0
0.09955622619051102 -6.429046385479891e-17 0
0.09955622619051102 -6.429046385479891e-17 0
0.09955622619051102 -6.429046385479891e-17 0
'''

def split_modes(input_data, columns_to_remove=None):
    data_arrays = []

    # Split the input data into individual arrays
    arrays = input_data.strip().split('\n\n')

    for array in arrays:
        # Splitting the data into rows and then into elements, converting each to a float
        array = [list(map(float, row.split())) for row in array.strip().split('\n')]
        # Converting the data to a NumPy matrix
        data_arrays.append(np.array(array))
    return data_arrays

GM5m_eigenvec_super = split_modes(GM5m_eigenvec_super)

In [20]:
for i in range(len(GM5m_eigenvec_super)):
    #moded_matrix = add_distortions_by_labels(np.array([row[:] for row in add_numbers2labels(labeled_matrix)]), np.array(result[i]['data']), lattice_basis) #labled matrix had columns of strings - convert to float    
    #    moded_matrix = add_distortions_by_labels(moded_matrix, np.array(result[i+1]['data']),lattice_basis)


    def matrix_to_string(matrix):
        # Update this function to handle labels correctly
        row_strings = [" ".join(str(value) for value in row) for row in matrix]
        return "\n".join(row_strings)

    diff = np.array(I2cm-P4mmm_matrix)

    #convert to cart. coordinates
    cart_diff = np.dot(diff,np.linalg.inv(lattice_basis)).flatten()

    mode = np.array(GM5m_eigenvec_super[i].flatten())

    print(np.dot(mode, cart_diff))

0.07968639122570369
-0.02533884980594506
-0.055174594021899934
-0.16576411458428714
0.03684374350844148
0.03684374350844148
0.002507441488618103
-0.05181806235437338
-0.11802188089628596
0.08644326398912762
0.01604039525440884
0.1021359472344202
-0.009993689252570579
-0.03581672903666089
0.08137614212517603


In [17]:
GM5m_eigenvec = '''
0.01332460965893595 0.01332460965893595 0
0.5727051292669996 0.5727051292669996 0
0.04725483944561602 0.04725483944561602 0
0.04725483944561602 0.04725483944561602 0
-0.04016363402578949 -0.04016665969147076 0
-0.04016665969147076 -0.04016363402578949 0
-0.04016665969147076 -0.04016363402578949 0
-0.04016363402578949 -0.04016665969147076 0
-0.06417280241913655 -0.06417280241913655 0
-0.06417280241913655 -0.06417280241913655 0
-0.3907377595627762 -0.3907377595627762 0

0.2163225644205847 0.2163225644205847 0
0.2126239130622329 0.2126239130622329 0
0.2128702838988771 0.2128702838988771 0
0.2128702838988771 0.2128702838988771 0
0.2128520464587878 0.2128689086001777 0
0.2128689086001777 0.2128520464587878 0
0.2128689086001777 0.2128520464587878 0
0.2128520464587878 0.2128689086001777 0
0.2130190022880627 0.2130190022880627 0
0.2130190022880627 0.2130190022880627 0
0.2130154493222874 0.2130154493222874 0

-0.6729165493800852 -0.6729165493800852 0
0.0840125175129704 0.0840125175129704 0
0.06055805863269077 0.06055805863269077 0
0.06055805863269077 0.06055805863269077 0
0.06769853034045115 0.06770398099161626 0
0.06770398099161626 0.06769853034045115 0
0.06770398099161626 0.06769853034045115 0
0.06769853034045115 0.06770398099161626 0
0.07230558919200251 0.07230558919200251 0
0.07230558919200251 0.07230558919200251 0
0.06325046228499638 0.06325046228499638 0

0.007796509943485155 0.007796509943485155 0
-0.02851103362369972 -0.02851103362369972 0
-0.1049117481684309 -0.1049117481684309 0
-0.1049117481684309 -0.1049117481684309 0
-0.1230982863218575 -0.1231093263485936 0
-0.1231093263485936 -0.1230982863218575 0
-0.1231093263485936 -0.1230982863218575 0
-0.1230982863218575 -0.1231093263485936 0
0.4418378639887262 0.4418378639887262 0
0.4418378639887262 0.4418378639887262 0
-0.1614118980121724 -0.1614118980121724 0

-0.00858398966611257 -0.00858398966611257 0
-0.3442817475445299 -0.3442817475445299 0
0.2115981924005003 0.2115981924005003 0
0.2115981924005003 0.2115981924005003 0
0.104002631209924 0.1040156447822053 0
0.1040156447822053 0.104002631209924 0
0.1040156447822053 0.104002631209924 0
0.104002631209924 0.1040156447822053 0
0.006112178378409781 0.006112178378409781 0
0.006112178378409781 0.006112178378409781 0
-0.4985002333095033 -0.4985002333095033 0

-0.00870919193683301 -0.00870919193683301 0
-0.01997505132699418 -0.01997505132699418 0
0.3781609092072056 0.3781609092072056 0
0.3781609092072056 0.3781609092072056 0
-0.2179582572658199 -0.2180292157330482 0
-0.2180292157330482 -0.2179582572658199 0
-0.2180292157330482 -0.2179582572658199 0
-0.2179582572658199 -0.2180292157330482 0
-0.0043128977064078 -0.0043128977064078 0
-0.0043128977064078 -0.0043128977064078 0
0.152942855260243 0.152942855260243 0

-5.087228025219472e-07 -5.087228025219472e-07 0
6.191011969862439e-07 6.191011969862439e-07 0
2.707552790150103e-05 2.707552790150103e-05 0
2.707552790150103e-05 2.707552790150103e-05 0
-0.3535848703007139 0.3535219068910089 0
0.3535219068910089 -0.3535848703007139 0
0.3535219068910089 -0.3535848703007139 0
-0.3535848703007139 0.3535219068910089 0
4.413610936245959e-07 4.413610936245959e-07 0
4.413610936245959e-07 4.413610936245959e-07 0
1.476293737835838e-05 1.476293737835838e-05 0
'''

def split_modes(input_data, columns_to_remove=None):
    data_arrays = []

    # Split the input data into individual arrays
    arrays = input_data.strip().split('\n\n')

    for array in arrays:
        # Splitting the data into rows and then into elements, converting each to a float
        array = [list(map(float, row.split())) for row in array.strip().split('\n')]
        # Converting the data to a NumPy matrix
        data_arrays.append(np.array(array))
    return data_arrays

GM5m_eigenvec = split_modes(GM5m_eigenvec)
print(GM5m_eigenvec)
for i in range(len(GM5m_eigenvec)):
    #moded_matrix = add_distortions_by_labels(np.array([row[:] for row in add_numbers2labels(labeled_matrix)]), np.array(result[i]['data']), lattice_basis) #labled matrix had columns of strings - convert to float    
    #    moded_matrix = add_distortions_by_labels(moded_matrix, np.array(result[i+1]['data']),lattice_basis)


    def matrix_to_string(matrix):
        # Update this function to handle labels correctly
        row_strings = [" ".join(str(value) for value in row) for row in matrix]
        return "\n".join(row_strings)
    
    #print(matrix_to_string(moded_matrix[:,1:]), '\n') 

    diff = np.array(GM5m_dist-P4mmm_prim).flatten()
    mode = np.array(GM5m_eigenvec[i].flatten())
    print(np.dot(mode, diff))


[array([[ 0.01332461,  0.01332461,  0.        ],
       [ 0.57270513,  0.57270513,  0.        ],
       [ 0.04725484,  0.04725484,  0.        ],
       [ 0.04725484,  0.04725484,  0.        ],
       [-0.04016363, -0.04016666,  0.        ],
       [-0.04016666, -0.04016363,  0.        ],
       [-0.04016666, -0.04016363,  0.        ],
       [-0.04016363, -0.04016666,  0.        ],
       [-0.0641728 , -0.0641728 ,  0.        ],
       [-0.0641728 , -0.0641728 ,  0.        ],
       [-0.39073776, -0.39073776,  0.        ]]), array([[0.21632256, 0.21632256, 0.        ],
       [0.21262391, 0.21262391, 0.        ],
       [0.21287028, 0.21287028, 0.        ],
       [0.21287028, 0.21287028, 0.        ],
       [0.21285205, 0.21286891, 0.        ],
       [0.21286891, 0.21285205, 0.        ],
       [0.21286891, 0.21285205, 0.        ],
       [0.21285205, 0.21286891, 0.        ],
       [0.213019  , 0.213019  , 0.        ],
       [0.213019  , 0.213019  , 0.        ],
       [0.21301545,

In [28]:
import numpy as np
def split_modes(input_data, columns_to_remove=None):
    data_arrays = []

    # Split the input data into individual arrays
    arrays = input_data.strip().split('\n\n')

    for array in arrays:
        lines = array.split('\n')

        # Remove the first line
        header = lines.pop(0)

        data = []
        for line in lines:
            if line.strip():  # Check if the line is not empty
                components = line.split()
                # Skip the first component and convert the rest to floats
                row = [components[0]] + [float(component) for component in components[1:]]
                
                data.append(row)

        # Remove specific columns if specified
        if columns_to_remove:
            for row in data:
                for col_index in sorted(columns_to_remove, reverse=True):
                    # Adjusted to account for the skipped first column
                    if col_index != 0:
                        del row[col_index]

        # Construct the data array dictionary
        data_arrays.append({'header': header, 'data': data})

    return data_arrays

def add_distortions_by_labels(matrix1, matrix2, lattice_basis):
    """
    Adds distortions to rows in matrix1 based on corresponding rows in matrix2.

    Args:
    - matrix1: First array with rows to be distorted
    - matrix2: Second array with distortions and labels
    - lattice_basis: Basis matrix for transformations

    Returns:
    - result_matrix: Matrix1 with distortions added
    """


    matrix_copy = np.copy(matrix1[:, 1:].astype(float))

    shape_matrix1 = matrix1[:, 1:].astype(float).shape
    # Create a new empty matrix of the same shape as matrix1
    empty_matrix_pos = np.zeros(shape_matrix1)

    # Convert matrix2 to Cartesian coordinates
    cartesian_matrix2 = np.dot(matrix2[:, 1:].astype(float), lattice_basis)

    # Normalize entire matrix2   
    normalized_matrix2 = cartesian_matrix2 / np.linalg.norm(cartesian_matrix2)#, axis=1)[:, np.newaxis]

    for row, normalized_row in zip(matrix2, normalized_matrix2):
        label = row[0]
        for i, row1 in enumerate(matrix1):
            if row1[0] == label:
                inverse_t = np.linalg.inv(lattice_basis)

                direct = np.dot(inverse_t, normalized_row)
                
                distortion = direct * 0.1

                matrix_copy[i] += distortion
                empty_matrix_pos[i] += normalized_row
                
    return np.hstack((matrix1[:, :1], empty_matrix_pos))

def add_numbers2labels(data):
    data = np.array(data)

    # Dictionary to keep track of label counts
    label_counts = {}

    # Iterate through the array and append numbers to labels
    for i, row in enumerate(data):
        label = row[0]
        if label not in label_counts:
            label_counts[label] = 1
        else:
            label_counts[label] += 1
        data[i][0] = f"{label}{label_counts[label]}"

    return data


# Process the data with specified columns to remove
result = split_modes(A5minus, [1,2,3])




for i in range(len(result)):
    moded_matrix = add_distortions_by_labels(np.array([row[:] for row in add_numbers2labels(labeled_matrix)]), np.array(result[i]['data']), lattice_basis) #labled matrix had columns of strings - convert to float    
    #    moded_matrix = add_distortions_by_labels(moded_matrix, np.array(result[i+1]['data']),lattice_basis)


    def matrix_to_string(matrix):
        # Update this function to handle labels correctly
        row_strings = [" ".join(str(value) for value in row) for row in matrix]
        return "\n".join(row_strings)
    
    #print(matrix_to_string(moded_matrix[:,1:]), '\n') 

    diff = np.array(A5minus_dist-P4mmm_dist).flatten()
    mode = np.array(moded_matrix[:,1:].astype(float)).flatten()
    print(np.dot(mode, diff))

    
    #formatted_unlabeled = matrix_to_string(moded_matrix[:,1:].astype(float) + A5minus_dist*0.1)# + A5minus_dist*0.1)
    #print(formatted_unlabeled, '\n')


-0.4796886054078442
-0.4999999999901519
0.6931546173552185
-0.7071067811778858
-0.04772143624575174
0.0
0.5311253370028851
0.0
-0.35852614044772224
0.499999999992923


In [18]:
mode = split_modes(A2plus, [1,2,3])
np.array(mode[0]['data'])

NameError: name 'A2plus' is not defined

In [8]:
GM5m_dist = '''
-4.282264356115649e-13 -0.01257767725184052 0
-4.282264356115649e-13 -0.01257767725184052 0
-4.282264356115649e-13 -0.01257767725184052 0
-4.282264356115649e-13 -0.01257767725184052 0
-1.384181624982011e-11 -0.4069754990069022 0
-1.384181624982011e-11 -0.4069754990069022 0
-1.384181624982011e-11 -0.4069754990069022 0
-1.384181624982011e-11 -0.4069754990069022 0
-1.325659925579004e-12 -0.03897550780621601 0
-1.325659925579004e-12 -0.03897550780621601 0
-1.325659925579004e-12 -0.03897550780621601 0
-1.325659925579004e-12 -0.03897550780621601 0
-1.325659925579004e-12 -0.03897550780621601 0
-1.325659925579004e-12 -0.03897550780621601 0
-1.325659925579004e-12 -0.03897550780621601 0
-1.325659925579004e-12 -0.03897550780621601 0
-0.01096309476322607 0.03790457904264066 0
-0.01096309476322607 0.03790457904264066 0
-0.01096309476322607 0.03790457904264066 0
-0.01096309476322607 0.03790457904264066 0
-0.01096309476322607 0.03790457904264066 0
-0.01096309476322607 0.03790457904264066 0
-0.01096309476322607 0.03790457904264066 0
-0.01096309476322607 0.03790457904264066 0
0.01096309476580403 0.03790457904338629 0
0.01096309476580403 0.03790457904338629 0
0.01096309476580403 0.03790457904338629 0
0.01096309476580403 0.03790457904338629 0
0.01096309476580403 0.03790457904338629 0
0.01096309476580403 0.03790457904338629 0
0.01096309476580403 0.03790457904338629 0
0.01096309476580403 0.03790457904338629 0
1.336972852490332e-12 0.03931571295173966 0
1.336972852490332e-12 0.03931571295173966 0
1.336972852490332e-12 0.03931571295173966 0
1.336972852490332e-12 0.03931571295173966 0
1.336972852490332e-12 0.03931571295173966 0
1.336972852490332e-12 0.03931571295173966 0
1.336972852490332e-12 0.03931571295173966 0
1.336972852490332e-12 0.03931571295173966 0
9.116876177635958e-12 0.26805844142415 0
9.116876177635958e-12 0.26805844142415 0
9.116876177635958e-12 0.26805844142415 0
9.116876177635958e-12 0.26805844142415 0
'''

# Splitting the data into rows and then into elements, converting each to a float
GM5m_dist = [list(map(float, row.split())) for row in GM5m_dist.strip().split('\n')]

# Converting the data to a NumPy matrix
GM5m_dist = np.array(GM5m_dist)

### Tuning the direction of modes

In [16]:
def direction_tuning(eigenvector, modes, direction='a0'):
    coef_SAM = []
    for i in range(len(modes)):
        coef=np.dot(eigenvector.flatten(),modes[i].flatten())
        coef_SAM.append(coef)
    #print(coef_SAM)

    # Initialize a variable to store the larger number
    large_number = None

    # Iterate over the list
    for i, num in enumerate(coef_SAM):
        # Check if the number is close to zero
        if abs(num) < 1e-8:  # You can adjust the threshold as needed
            # Replace the small number with the preceding larger number
            if direction == 'a0':
                coef_SAM[i] = 0 #coef_SAM[i-1]
            else: coef_SAM[i] = coef_SAM[i-1]
        #else:
            # Update the larger number
            #print('No small numbers found')
    coef_SAM = coef_SAM/np.linalg.norm(coef_SAM)

    GM5minus_eigenvec = np.zeros((44,3))

    for i in range(len(modes)):
        new_vector=np.dot(modes[i].flatten(),coef_SAM[i])
        new_vector= new_vector.reshape(44,3)
        GM5minus_eigenvec+=new_vector
    return GM5minus_eigenvec

In [17]:
for i in range(len(GM5m_eigenvec_super)):
    GM5m_eigenvec_fixed = direction_tuning(GM5m_eigenvec_super[i], GM5_modes)
    diff = np.array(I2cm-P4mmm_matrix).flatten()
    mode = GM5m_eigenvec_fixed.flatten()
    print(np.dot(mode, diff))    

0.4347867217407819
-0.1382544154535355
-0.3010448897469766
-0.9044459769354953
0.2010276812661945
0.2010276812661945
0.013681159947430554
-0.2827309043777684
-0.6439537027426517
0.4716537264130003
0.08751997374581341
0.5572765058946031
-0.054527797279178586
-0.195424061268521
0.4440063794822971


In [23]:
np.linalg.norm([0.4347867217407819,-0.1382544154535355,-0.3010448897469766,-0.9044459769354953,0.2010276812661945,0.013681159947430554,-0.2827309043777684,-0.6439537027426517,0.4716537264130003,0.08751997374581341,0.5572765058946031,-0.054527797279178586,-0.195424061268521,0.4440063794822971])

1.559165277594615

In [9]:
GM5_modes = '''
0.0 0.5 0.0
0.0 0.5 0.0
0.0 0.5 0.0
0.0 0.5 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0 

0.5 0.0 0.0
0.5 0.0 0.0
0.5 0.0 0.0
0.5 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0 

0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.5 0.0
0.0 0.5 0.0
0.0 0.5 0.0
0.0 0.5 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0 

0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.5 0.0 0.0
0.5 0.0 0.0
0.5 0.0 0.0
0.5 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0 

0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.35355339059327373 0.0
0.0 0.35355339059327373 0.0
0.0 0.35355339059327373 0.0
0.0 0.35355339059327373 0.0
0.0 0.35355339059327373 0.0
0.0 0.35355339059327373 0.0
0.0 0.35355339059327373 0.0
0.0 0.35355339059327373 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0 

0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.35355339059327373 0.0 0.0
0.35355339059327373 0.0 0.0
0.35355339059327373 0.0 0.0
0.35355339059327373 0.0 0.0
0.35355339059327373 0.0 0.0
0.35355339059327373 0.0 0.0
0.35355339059327373 0.0 0.0
0.35355339059327373 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0 

0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
-0.17677669529663687 0.17677669529663687 0.0
-0.17677669529663687 0.17677669529663687 0.0
-0.17677669529663687 0.17677669529663687 0.0
-0.17677669529663687 0.17677669529663687 0.0
-0.17677669529663687 0.17677669529663687 0.0
-0.17677669529663687 0.17677669529663687 0.0
-0.17677669529663687 0.17677669529663687 0.0
-0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0 

0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.17677669529663687 -0.17677669529663687 0.0
0.17677669529663687 -0.17677669529663687 0.0
0.17677669529663687 -0.17677669529663687 0.0
0.17677669529663687 -0.17677669529663687 0.0
0.17677669529663687 -0.17677669529663687 0.0
0.17677669529663687 -0.17677669529663687 0.0
0.17677669529663687 -0.17677669529663687 0.0
0.17677669529663687 -0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0 

0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
-0.17677669529663687 0.17677669529663687 0.0
-0.17677669529663687 0.17677669529663687 0.0
-0.17677669529663687 0.17677669529663687 0.0
-0.17677669529663687 0.17677669529663687 0.0
-0.17677669529663687 0.17677669529663687 0.0
-0.17677669529663687 0.17677669529663687 0.0
-0.17677669529663687 0.17677669529663687 0.0
-0.17677669529663687 0.17677669529663687 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0 

0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 0.17677669529663687 0.0
0.17677669529663687 -0.17677669529663687 0.0
0.17677669529663687 -0.17677669529663687 0.0
0.17677669529663687 -0.17677669529663687 0.0
0.17677669529663687 -0.17677669529663687 0.0
0.17677669529663687 -0.17677669529663687 0.0
0.17677669529663687 -0.17677669529663687 0.0
0.17677669529663687 -0.17677669529663687 0.0
0.17677669529663687 -0.17677669529663687 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0 

0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.35355339059327373 0.0
0.0 0.35355339059327373 0.0
0.0 0.35355339059327373 0.0
0.0 0.35355339059327373 0.0
0.0 0.35355339059327373 0.0
0.0 0.35355339059327373 0.0
0.0 0.35355339059327373 0.0
0.0 0.35355339059327373 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0 

0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.35355339059327373 0.0 0.0
0.35355339059327373 0.0 0.0
0.35355339059327373 0.0 0.0
0.35355339059327373 0.0 0.0
0.35355339059327373 0.0 0.0
0.35355339059327373 0.0 0.0
0.35355339059327373 0.0 0.0
0.35355339059327373 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0 

0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.5 0.0
0.0 0.5 0.0
0.0 0.5 0.0
0.0 0.5 0.0 

0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.0 0.0 0.0
0.5 0.0 0.0
0.5 0.0 0.0
0.5 0.0 0.0
0.5 0.0 0.0 
'''
def split_modes(input_data, columns_to_remove=None):
    data_arrays = []

    # Split the input data into individual arrays
    arrays = input_data.strip().split('\n\n')

    for array in arrays:
        # Splitting the data into rows and then into elements, converting each to a float
        array = [list(map(float, row.split())) for row in array.strip().split('\n')]
        # Converting the data to a NumPy matrix
        data_arrays.append(np.array(array))
    return data_arrays

GM5_modes = split_modes(GM5_modes)

In [22]:
for i in range(len(GM5_modes)):
    #moded_matrix = add_distortions_by_labels(np.array([row[:] for row in add_numbers2labels(labeled_matrix)]), np.array(result[i]['data']), lattice_basis) #labled matrix had columns of strings - convert to float    
    #    moded_matrix = add_distortions_by_labels(moded_matrix, np.array(result[i+1]['data']),lattice_basis)


    def matrix_to_string(matrix):
        # Update this function to handle labels correctly
        row_strings = [" ".join(str(value) for value in row) for row in matrix]
        return "\n".join(row_strings)
    
    #print(matrix_to_string(moded_matrix[:,1:]), '\n') 

    #convert to cart. coordinates
    #cart_I2cm = np.dot(I2cm,np.linalg.inv(I2cm_lattice))
    #cart_P4mmm_matrix = np.dot(P4mmm_matrix,np.linalg.inv(lattice_basis))

    diff = np.array(I2cm-P4mmm_matrix)
    cart_diff = np.dot(diff,np.linalg.inv(I2cm_lattice)).flatten()
    
    mode = np.array(GM5_modes[i].flatten())

    print(np.dot(cart_diff,mode))

0.006603496360629096
0.0
0.01056585634251657
0.06545314732985848
0.0009321024603532319
0.09256472865389018
-0.011201742151726701
-8.673617379884035e-19
-0.0026486161710513353
-1.3010426069826053e-18
0.0049089330425247085
0.09256472865389018
0.12437439619052086
0.06545314732985848


In [7]:
coef_SAM = []
for i in range(len(GM5_modes)):
    coef=np.dot(GM5m_dist.flatten(),GM5_modes[i].flatten())
    coef_SAM.append(coef)
print(coef_SAM)

# Initialize a variable to store the larger number
large_number = None

# Iterate over the list
for i, num in enumerate(coef_SAM):
    # Check if the number is close to zero
    if abs(num) < 1e-8:  # You can adjust the threshold as needed
        # Replace the small number with the preceding larger number
        coef_SAM[i] = 0 #coef_SAM[i-1]
    else:
        # Update the larger number
        print('No small numbers found')
coef_SAM = coef_SAM/np.linalg.norm(coef_SAM)

print(coef_SAM)

NameError: name 'GM5m_dist' is not defined

In [21]:
GM5minus_eigenvec = np.zeros((44,3))
for i in range(len(GM5_modes)):
    new_vector=np.dot(GM5_modes[i].flatten(),coef_SAM[i])
    new_vector= new_vector.reshape(44,3)
    GM5minus_eigenvec+=new_vector
    #print(new_vector)
#print(matrix_to_string(unlabled_matrix +eigenvec*0.1))    
for i in [-0.05,-0.03,-0.01,-0.005,0.005,0.01,0.03,0.05]:
   print(matrix_to_string(P4mmm_matrix+GM5minus_eigenvec*i), '\n')


0.0 0.0006288838625920258 0.25
0.5 0.5006288838625921 0.25
0.0 0.0006288838625920258 0.75
0.5 0.5006288838625921 0.75
0.0 0.020348774950345108 0.0
0.5 0.5203487749503451 0.0
0.0 0.020348774950345108 0.5
0.5 0.5203487749503451 0.5
0.5 0.0019487753903108 0.1002905922092587
0.0 0.5019487753903108 0.1002905922092587
0.5 0.0019487753903108 0.6002905922092587
0.0 0.5019487753903108 0.6002905922092587
0.5 0.0019487753903108 0.3997093927907399
0.0 0.5019487753903108 0.3997093927907399
0.5 0.0019487753903108 0.8997093927907399
0.0 0.5019487753903108 0.8997093927907399
0.2505481547382257 0.7481047710478493 0.08044711925346695
0.7505481547382258 0.24810477104784934 0.08044711925346695
0.2505481547382257 0.7481047710478493 0.5804471192534669
0.7505481547382258 0.24810477104784934 0.5804471192534669
0.2505481547382257 0.7481047710478493 0.41955288074653296
0.7505481547382258 0.24810477104784934 0.41955288074653296
0.2505481547382257 0.7481047710478493 0.919552880746533
0.7505481547382258 0.24810477

In [115]:
A2plus = '''
P4/mmm[1/2,1/2,1/2]A2+(a)[O1:i:dsp]B2(a) normfactor =  0.03199
O3    0.25005  0.75005  0.58143 -1.0000  1.0000  0.0000 
O1    0.25005  0.75005  0.08143  1.0000 -1.0000  0.0000 
O4    0.75005  0.25005  0.58143  1.0000 -1.0000  0.0000 
O2    0.75005  0.25005  0.08143 -1.0000  1.0000  0.0000 
O6    0.75005  0.25005  0.41867  1.0000 -1.0000  0.0000 
O8    0.75005  0.25005  0.91867 -1.0000  1.0000  0.0000 
O5    0.25005  0.75005  0.41867 -1.0000  1.0000  0.0000 
O7    0.25005  0.75005  0.91867  1.0000 -1.0000  0.0000 
O13   0.25005  0.25005  0.41867  1.0000  1.0000  0.0000 
O15   0.25005  0.25005  0.91867 -1.0000 -1.0000  0.0000 
O14   0.75005  0.75005  0.41867 -1.0000 -1.0000  0.0000 
O16   0.75005  0.75005  0.91867  1.0000  1.0000  0.0000 
O11   0.25005  0.25005  0.58143  1.0000  1.0000  0.0000 
O9    0.25005  0.25005  0.08143 -1.0000 -1.0000  0.0000 
O12   0.75005  0.75005  0.58143 -1.0000 -1.0000  0.0000 
O10   0.75005  0.75005  0.08143  1.0000  1.0000  0.0000 
'''

In [116]:
A5minus_dist = """
0 0 0
0 0 0
0 0 0
0 0 0
0.2045219399607055 6.722480503321745e-15 0
-0.2045219399607055 -6.722480503321745e-15 0
-0.2045219399607055 -6.722480503321745e-15 0
0.2045219399607055 6.722480503321745e-15 0
0.02571228125277871 -8.475620083108975e-16 0
-0.02571228125277871 8.475620083108975e-16 0
-0.02571228125277871 8.475620083108975e-16 0
0.02571228125277871 -8.475620083108975e-16 0
-0.02571228125277871 8.475620083108975e-16 0
0.02571228125277871 -8.475620083108975e-16 0
0.02571228125277871 -8.475620083108975e-16 0
-0.02571228125277871 8.475620083108975e-16 0
0 0 0.133151399655914
0 0 -0.133151399655914
0 0 -0.133151399655914
0 0 0.133151399655914
0 0 0.133151399655914
0 0 -0.133151399655914
0 0 -0.133151399655914
0 0 0.133151399655914
0 0 0.1331513996559229
0 0 -0.1331513996559229
0 0 -0.1331513996559229
0 0 0.1331513996559229
0 0 0.1331513996559229
0 0 -0.1331513996559229
0 0 -0.1331513996559229
0 0 0.1331513996559229
0.07660672759536045 -2.582096897444975e-15 0
-0.07660672759536045 2.582096897444975e-15 0
-0.07660672759536045 2.582096897444975e-15 0
0.07660672759536045 -2.582096897444975e-15 0
-0.07660672759536045 2.582096897444975e-15 0
0.07660672759536045 -2.582096897444975e-15 0
0.07660672759536045 -2.582096897444975e-15 0
-0.07660672759536045 2.582096897444975e-15 0
-0.3524119335567476 1.173952250160603e-14 0
0.3524119335567476 -1.173952250160603e-14 0
0.3524119335567476 -1.173952250160603e-14 0
-0.3524119335567476 1.173952250160603e-14 0
"""
# Splitting the data into rows and then into elements, converting each to a float
A5minus_dist = [list(map(float, row.split())) for row in A5minus_dist.strip().split('\n')]

# Converting the data to a NumPy matrix
A5minus_dist = np.array(A5minus_dist)

In [29]:
import numpy as np
def split_modes(input_data, columns_to_remove=None):
    data_arrays = []

    # Split the input data into individual arrays
    arrays = input_data.strip().split('\n\n')

    for array in arrays:
        lines = array.split('\n')

        # Remove the first line
        header = lines.pop(0)

        data = []
        for line in lines:
            if line.strip():  # Check if the line is not empty
                components = line.split()
                # Skip the first component and convert the rest to floats
                row = [components[0]] + [float(component) for component in components[1:]]
                
                data.append(row)

        # Remove specific columns if specified
        if columns_to_remove:
            for row in data:
                for col_index in sorted(columns_to_remove, reverse=True):
                    # Adjusted to account for the skipped first column
                    if col_index != 0:
                        del row[col_index]

        # Construct the data array dictionary
        data_arrays.append({'header': header, 'data': data})

    return data_arrays

def add_distortions_by_labels(matrix1, matrix2, lattice_basis):
    """
    Adds distortions to rows in matrix1 based on corresponding rows in matrix2.

    Args:
    - matrix1: First array with rows to be distorted
    - matrix2: Second array with distortions and labels
    - lattice_basis: Basis matrix for transformations

    Returns:
    - result_matrix: Matrix1 with distortions added
    """
    
    # rotation_matrix = np.array([
    #                                 [1, -1, 0],
    #                                 [1, 1, 0],
    #                                 [0, 0, 1]
    #                             ])
    # direct2cart = np.dot(np.linalg.inv(rotation_matrix),lattice_basis)


    matrix_copy = np.copy(matrix1[:, 1:].astype(float))

    #print(matrix2)
    #print(lattice_basis)
    # Convert matrix2 to Cartesian coordinates
    cartesian_matrix2 = np.dot(matrix2[:, 1:].astype(float), lattice_basis)

    # Normalize entire matrix2   
    normalized_matrix2 = cartesian_matrix2 / np.linalg.norm(cartesian_matrix2)#, axis=1)[:, np.newaxis]
    #print(normalized_matrix2)
    for row, normalized_row in zip(matrix2, normalized_matrix2):
        label = row[0]
        for i, row1 in enumerate(matrix1):
            if row1[0] == label:
                inverse_t = np.linalg.inv(lattice_basis)

                direct = np.dot(inverse_t, normalized_row)
                
                direct = direct/np.linalg.norm(inverse_t)
                distortion = direct * 0.1
                #print(distortion)

                matrix_copy[i] += distortion

    return np.hstack((matrix1[:, :1], matrix_copy))

def add_numbers2labels(data):
    data = np.array(data)

    # Dictionary to keep track of label counts
    label_counts = {}

    # Iterate through the array and append numbers to labels
    for i, row in enumerate(data):
        label = row[0]
        if label not in label_counts:
            label_counts[label] = 1
        else:
            label_counts[label] += 1
        data[i][0] = f"{label}{label_counts[label]}"

    return data


# Process the data with specified columns to remove
result = split_modes(A2plus, [1,2,3])
for i in range(len(result)):
    moded_matrix = add_distortions_by_labels(np.array([row[:] for row in add_numbers2labels(labeled_matrix)]), np.array(result[i]['data']), lattice_basis) #labled matrix had columns of strings - convert to float    
    #    moded_matrix = add_distortions_by_labels(moded_matrix, np.array(result[i+1]['data']),lattice_basis)
    def matrix_to_string(matrix):
        # Update this function to handle labels correctly
        row_strings = [" ".join(str(value) for value in row) for row in matrix]
        return "\n".join(row_strings)
    #labels = np.array(labeled_matrix[:, 0]).reshape(-1, 1) 
    #labeled_moded_positions = np.hstack([labels, np.array(moded_matrix[:,1:])])

    # Convert the matrix to a string in the desired format
    #formatted_labeled = matrix_to_string(labeled_moded_positions)
    #print(moded_matrix)
    A2p_dist = moded_matrix[:,1:].astype(float) - P4mmm_matrix
    print(A2p_dist)
    formatted_unlabeled = matrix_to_string(A5minus_eigenvec[0]*0.1 + P4mmm_matrix+ GM5m_eigenvec_fixed[0]*0.1 + A2p_dist)#moded_matrix[:,1:].astype(float))# + A5minus_dist*0.1)#+ GM5minus_eigenvec*0.1)# + A5minus_dist*0.1)
    print(formatted_unlabeled, '\n')
    #print(formatted_labeled)

[[ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.01231558 -0.01231558  0.        ]
 [-0.01231558  0.01231558  0.        ]
 [-0.01231558  0.01231558  0.        ]
 [ 0.01231558 -0.01231558  0.        ]
 [-0.01231558  0.01231558  0.        ]
 [ 0.01231558 -0.01231558  0.        ]
 [ 0.01231558 -0.01231558  0.        ]
 [-0.01231558  0.01231558  0.        ]
 [-0.01231558 -0.01231558  0.        ]
 [ 0.01231558  0.01231558

In [20]:
dyn_matrix = '''
  0.146404560298   0.000000000000   0.000079899020
  1.198508045932   0.000000000000   0.000340276516
 -9.066219229211   0.000000000000  -0.000781386385
 -0.005794202452   0.000000000000  -0.000725139197
  9.594732721036   0.000000000000   0.000914040786
 -0.035664148393   0.000000000000   0.000328352950
 -1.982842828394   0.000000000000  -0.000138560501
 -0.004723107485   0.000000000000   0.000284376233
  0.408958428862   0.000000000000   0.000187221358
  0.030265928303   0.000000000000  -0.000233332678
 -0.283626168495   0.000000000000  -0.000255748100
'''
# Splitting the data into rows and then into elements, converting each to a float
dyn_matrix = [list(map(float, row.split())) for row in dyn_matrix.strip().split('\n')]

# Converting the data to a NumPy matrix
dyn_matrix = np.array(dyn_matrix)*np.sqrt( 85.468*144.240 *92.910 *16.000)
FCM_matrix= dyn_matrix/np.linalg.norm(dyn_matrix)
FCM_matrix

array([[ 1.09155453e-02,  0.00000000e+00,  5.95706427e-06],
       [ 8.93576599e-02,  0.00000000e+00,  2.53701369e-05],
       [-6.75953855e-01,  0.00000000e+00, -5.82581478e-05],
       [-4.32000748e-04,  0.00000000e+00, -5.40645029e-05],
       [ 7.15358454e-01,  0.00000000e+00,  6.81485168e-05],
       [-2.65902666e-03,  0.00000000e+00,  2.44811466e-05],
       [-1.47835633e-01,  0.00000000e+00, -1.03307126e-05],
       [-3.52142678e-04,  0.00000000e+00,  2.12023564e-05],
       [ 3.04908827e-02,  0.00000000e+00,  1.39587402e-05],
       [ 2.25654933e-03,  0.00000000e+00, -1.73966809e-05],
       [-2.11464335e-02,  0.00000000e+00, -1.90679168e-05]])